# Parameter definitions and imports

## Parameters to be changed by user
It should be possible to run the code after setting only the variables in the cell below. In the provided directory names, '{group}' will be replaced by immediate/delay for the two groups. For example /my_data_dir/{group}/batch_data will be replaced by two directories, one called /my_data_dir/immediate/batch_data and one called /my_data_dir/delay/batch_data and the data from each group will be put in the corresponding directory.
File names have default names but can be changed as well in the code below
Assumes all directories exist

In [ ]:
#
# Locations of the files that have been downloaded from OSF
#

# The directory where the python code is placed (the file called stopmotion_funcs.py should be placed here)
code_dir = '/code_dir/'

# The directory where the R code is placed (the file called create_brms_models.R should be placed here)
R_dir = '/R_dir/'

# The directory holding all the logs of the experiment before processing (as saved from JATOS). The
# only processing the files have undergone is the replacement of the original participant ID from 
# prolific with an experiment-specific one (the contents of the 'logs' directory on OSF should be placed here)
log_dir = '/log_dir/logs/'

# File with the time of each action (relative to movie start), downloaded with experiment data
action_time_fname = '/downloaded_data_dir/action_timings.csv'

# Directory holding all the event segmentation logs, and the sub-directories for the neutral/
# surprise version. The event seg logs are in separate logs for each participant
seg_data_dir = '/seg_data_dir/'
seg_subdir = 'neutral/'
seg_S_subdir = 'surprise/'


#
# Empty directories that the code will place files in - these directories are expected to already exist
#

# A directory in which to place summary files for each batch (including overall subject performance, 
# the duration devoted to each section and a summary of the debriefing)
batch_summary_dir = '/my_data_dir/{group}/batch_summaries/'

# A directory in which to place more detailed files which are the result of the batch processing
batch_data_dir = '/my_data_dir/{group}/batch_data/'

# A directory in which to place files that are common to both groups
comb_data_dir = '/my_data_dir/combined/'
    
# A directory in which to put the brms models (for single-trial analysis). The data will also be saved here
brms_dir = '/brms_dir/'


#
# Additional params
#

# dpi used for plots (reduce for faster plotting)
plot_dpi = 75

# flag indicating whether to save the figures and a directory in which to save them 
# (can be left empty if flag set to false)
save_figs = False
fig_dir = '/fig_dir/'

# various figure size params (originally quite large for saving figures, left as reference)
fig_w = 6 #12
fig_h = 5 #10
label_font = 14 #28
tick_font = 12 #24
title_font = 16 #32


## Imports
System imports, import of the python code that processes logs (process_batch.py) and loading of rpy2 extension to allow incorporating R code in notebook. Adds also a dedicated R cell for imports in R, so that in the rest of the code the R cells are independent of one another.

In [ ]:
# All required imports

import sys
import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

sys.path.append(code_dir)
from stopmotion_funcs import process_batch, calc_surprise_dist, process_event_seg

%matplotlib inline
%load_ext rpy2.ipython

In [ ]:
%%R

library(dplyr)
library(plyr)
library(BayesFactor)
library(brms)
library(rslurm)
library(lme4)
library(lmerTest)

## Additional parameter definitions
These definitions do not require changing by user, as they are fixed for this experiment

In [ ]:

# Information regarding the debriefing questions - the total number of debrief questions and the first one
# including a survey (free text area)
n_debrief_q = 7
first_debrief_survey = 6

batch_n_subjects = 24 # Number os participants in each batch

batch_num = 8 # The batch number currently processed (8 for entire experiment)
max_n_batch = 8 # Maximal number of batches to run

# Criteria for subject exclusion - minimal accuracy (Pr) in validation questions 
# and minimal number of correct catch trials
validation_min_Pr = 0.75
catch_min_correct = 8

# Bayes Factor threshold for stopping the batches. Once this threshold is reached in favour of the 
# alternative (H1) for one of the tests or in favour of the null (H0) in all tests, that will indicate
# the criterion has been reached
bf_thresh = 6

# Names of the R script used for the single-trial analysis
create_models_fname = 'create_brms_models.R'

# Parallelisation parameters for the exploratory analysis
n_nodes = 10
cpus_per_node = 20
n_iter = 1000

#
# Event segmentation parameters
#

demo_boundaries = np.array([4042, 6708, 11042]) # timings of color-changes in demo
demo_length = 14000

# Timings of scene changes in each film
mov_scene_change =  [[12167, 51500, 69833, 105167, 122167],
        [10333, 29333, 40000, 58667, 71333, 87333, 92833, 103833, 131000, 
            152500, 164167, 182833, 200000, 213667, 230333, 240500, 262833],
        [11500, 46500, 56500, 74000, 106500, 128000, 169500, 189167, 216833, 
             230333, 260667, 268333, 304667, 333833, 350167]]
# Timings of target actions in each film
target_actions = [[29667, 78333, 138500],
        [20833, 43333, 80833, 117167, 158500, 188000, 223333, 274833],
        [29333, 88500, 151333, 208333, 238833, 313833, 355833]]

# Lengths of the three movies in msec
mov_lengths = np.array([152958, 289792, 376458])

# Minimal distance between button presses to be considered two boundaries
# (since long button presses can be logged multiple times)
boundary_min_dist = 2000

# Size of sliding window and jump (both in sec) for the boundary aggregation analysis
# (will calculate the number of people identifying a boundary in each win, at each jump)
seg_win_size = 3
seg_jump = 1

# Minimal number of participants that identified a boundary at each peak of the 
# sliding window analysis for that peak to be considered a boundary
boundary_min_subj = 10



# Data preparation

## Process log files
Takes the log files, as saved by JATOS, and extracts the information to a more convenient format for analysis. Also saves the formatted variables in files so they can be later loaded or opened in excel (saved as tab-separated files). There is one log file per batch of each group (immediate/delay).

In [ ]:
# Sets various 
# Organises the parameters in dictionaries as expected by the process_batch function. Also adds specific 
# filenames - default names are used for these, not set in parameters above.

exp_params = {
        'n_mov': len(mov_lengths),
        'n_debrief_q': n_debrief_q,
        'first_debrief_survey': first_debrief_survey,
        'batch_n_subjects': batch_n_subjects,
        'validation_min_Pr': validation_min_Pr,
        'catch_min_correct': catch_min_correct
}

file_params = {
        'results_temp': log_dir + '{group}_batch{batch_num}.txt',
        'summary_dir': batch_summary_dir,
        'debrief_temp': '{group}_batch{batch_num}_debrief.tsv',
        'performance_temp': '{group}_batch{batch_num}_performance.tsv',
        'durations_temp': '{group}_batch{batch_num}_durations.tsv',
        'data_dir': batch_data_dir,
        'comb_data_dir': comb_data_dir
}

# Loops over all batches until current one, loading the data from each. The process_batch
# function will load each batch if it has already been processed, or process the batch if 
# it hasn't. In either case it will return the same variables.
immediate_res = []
delay_res = []
for b in range(batch_num):
    immediate_res.append(process_batch('immediate', b+1, exp_params, file_params))
    delay_res.append(process_batch('delay', b+1, exp_params, file_params))

    
# Calculates the distance of each action from the preceding/following surprise, for each movie
# version. This function relys on the q_info_df and surprise_ver_df created by process_batch
# so must be run after process_batch has been run at least once. This df will be used to 
# run the secondary linear mixed model analysis
surprise_dist_df = calc_surprise_dist(action_time_fname, immediate_res[0]['surprise_ver_df'], 
                                      immediate_res[0]['q_info_df'], comb_data_dir)

#### Creates variables that hold the aggregated data, across all batches. One dataframe for the average performance and one with the surprise/neutral difference

In [ ]:
# Concatenates the immediate and delay groups of all batches, creating one full dataframe
avg_I = pd.concat([res['test_subj_avg_df'] for res in immediate_res])
avg_D = pd.concat([res['test_subj_avg_df'] for res in delay_res])

# The subject numbers are restarted in each batch for flexibility (e.g. concatenating 
# only dataframes of one group), so updates the subject number to run across all batches
# of both groups, then concatenates both groups into a single dataframe
avg_I['subj'] = avg_I['subj'] + (avg_I['batch']-1)*batch_n_subjects*2
avg_D['subj'] = avg_D['subj'] + (avg_D['batch']-1)*batch_n_subjects*2 + batch_n_subjects
full_subj_avg_df = pd.concat((avg_I, avg_D)).reset_index(drop=True)

# Creates a version of the dataframe with the Neutral Pr subtracted from the Surprise Pr of each subject,
# divide by question type
full_S_df = full_subj_avg_df[full_subj_avg_df['surprise_type']=='S'].drop(\
                                columns=['n_hit', 'n_HC_hit', 'avg_conf', 'n_FA', 'n_HC_FA', 
                                'avg_lure_conf', 'surprise_type', 'n_miss', 'n_HC_miss', 
                                'n_nFA', 'n_HC_nFA', 'd', 'HC_d']).\
                                rename({'Pr': 'Pr-S', 'HC_Pr': 'HC_Pr-S'}, axis=1)
full_N_df = full_subj_avg_df[full_subj_avg_df['surprise_type']=='N'].drop(\
                                columns=['n_hit', 'n_HC_hit', 'avg_conf', 'n_FA', 'n_HC_FA', 
                                'avg_lure_conf', 'surprise_type', 'n_miss', 'n_HC_miss', 
                                'n_nFA', 'n_HC_nFA', 'd', 'HC_d']).\
                                rename({'Pr': 'Pr-N', 'HC_Pr': 'HC_Pr-N'}, axis=1)
full_diff_df = full_S_df.merge(full_N_df, how='outer', validate='one_to_one')
full_diff_df['Pr_diff'] = full_diff_df['Pr-S']-full_diff_df['Pr-N']
full_diff_df['HC_Pr_diff'] = full_diff_df['HC_Pr-S']-full_diff_df['HC_Pr-N']

#### Counts and prints the number of participants excluded from each group

In [ ]:
# First across all batches
n_exc_I = full_subj_avg_df[(full_subj_avg_df['exc_subj'])&(full_subj_avg_df['group']=='I')]['subj'].nunique()
n_exc_D = full_subj_avg_df[(full_subj_avg_df['exc_subj'])&(full_subj_avg_df['group']=='D')]['subj'].nunique()
print(f'Number of excluded participants: {n_exc_I} in the immediate group and {n_exc_D} in the delay group')

# Then for the first batch only
n_exc_I_b1 = full_subj_avg_df[(full_subj_avg_df['exc_subj'])&(full_subj_avg_df['batch']==1)&(full_subj_avg_df['group']=='I')]['subj'].nunique()
n_exc_D_b1 = full_subj_avg_df[(full_subj_avg_df['exc_subj'])&(full_subj_avg_df['batch']==1)&(full_subj_avg_df['group']=='D')]['subj'].nunique()
print(f'Number of excluded participants in the first batch: {n_exc_I_b1} in the immediate group and {n_exc_D_b1} in the delay group')

## Process event-seg logs
Same as above, but here processes the event segmentation logs. There are two sets of logs, one group viewed the films with all scenes in their neutral version (presented in manuscript supplementary) and one group viewed the films with all scenes in their surprising version

In [ ]:
# Organises the parameters in dictionaries as expected by the process_event_seg function. Also adds specific 
# filenames - default names are used for these, not set in parameters above.

seg_params = {
    'demo_boundaries': demo_boundaries,
    'demo_length': demo_length,
    'mov_scene_change': mov_scene_change,
    'target_actions': target_actions,
    'mov_lengths': mov_lengths,
    'n_debrief_q': n_debrief_q,
    'first_debrief_survey': first_debrief_survey,
    'boundary_min_dist': boundary_min_dist,
    'win_size': seg_win_size,
    'boundary_min_subj': boundary_min_subj,
    'jump': seg_jump
}

seg_file_params = {
    'data_dir': seg_data_dir,
    'results_temp': seg_subdir + 'jatos_results_*.txt',
    'summary_fname': 'event_seg_performance_summary.tsv',
    'debrief_fname': 'event_seg_debrief.tsv',
    'durations_fname': 'event_seg_durations.tsv',
}

seg_S_file_params = {
    'data_dir': seg_data_dir,
    'results_temp': seg_S_subdir + 'jatos_results_*.txt',
    'summary_fname': 'event_seg_S_performance_summary.tsv',
    'debrief_fname': 'event_seg_S_debrief.tsv',
    'durations_fname': 'event_seg_S_durations.tsv',
}

seg_res = process_event_seg(seg_params, seg_file_params)
seg_S_res = process_event_seg(seg_params, seg_S_file_params)

# First batch calculations
Calculations from the 'Ceiling/floor performance' section of the manuscript

## First batch - data organisation
Organises the data (in python) for the R code that runs the actual calculations

In [ ]:
# Creates a separate dataframe for the first batch
first_batch_df = full_subj_avg_df[full_subj_avg_df['batch']==1]


# Creates a version of the first batch dataframe with the Target-N Pr subtracted from the Target-S Pr
target_diff_df = full_diff_df[(full_diff_df['batch']==1)&(full_diff_df['q_type']=='T')].\
                                    reset_index(drop=True)

## First batch - calculations (in R)
R code that calculates:
- Evidence (BF) in favour of no difference between accuracy on Target-S and Target-N questions
- Evidence (BF) in favour of zero accuracy on non-target questions, divided by group
- Number of participants with ceiling performance (Pr>0.9), divided by group


In [ ]:
%%R -i first_batch_df,target_diff_df

# Sets seed to arbitrary value at the beginning of each R cell, for reproducibility
set.seed(26)

# Removes excluded subjects from analysis
first_batch_data <- subset(first_batch_df, exc_subj=='FALSE')
target_diff_data <- subset(target_diff_df, exc_subj=='FALSE')

# Calculates the BF in favour of the null for a T-S vs. T-N comparison. (ttestBF gives the evidence
# in favour of the alternative - so derives the evidence in favour of the null). Runs a one-sided comparison,
# testing the evidence for T-S>T-N
target_bf <- ttestBF(as.numeric(unlist(target_diff_data['Pr_diff'])), nullInterval=c(0, Inf))
target_bf_H0 <- 1/as.numeric(as.vector(target_bf[1]))

# Calculates the evidence in favour of the null for accuracy in the non-target questions, 
# collapsing across surprising and neutral and across question type (since there's the 
# same number of events for each - simply averages the Pr from the subj average dataframe)

non_target_data <- subset(first_batch_data, q_type!='T')   
nontarget_comb_data_I <- subset(ddply(non_target_data, .(subj, group), summarise, mean_Pr = mean(Pr)),
                                group=='I')
nontarget_comb_data_D <- subset(ddply(non_target_data, .(subj, group), summarise, mean_Pr = mean(Pr)),
                                group=='D')
nontarget_bf_H0_I <- 1/as.numeric(as.vector(ttestBF(as.numeric(unlist(nontarget_comb_data_I['mean_Pr'])))))
nontarget_bf_H0_D <- 1/as.numeric(as.vector(ttestBF(as.numeric(unlist(nontarget_comb_data_D['mean_Pr'])))))

# Checks how many participants have a Pr>0.9 in the non-target questions
n_subj_ceiling_I <- sum(nontarget_comb_data_I['mean_Pr']>0.9)
n_subj_ceiling_D <- sum(nontarget_comb_data_D['mean_Pr']>0.9)


## First batch - display results


In [ ]:
# Pulls the relevant variables from the R code and converts to python native data types

%Rpull target_bf_H0 nontarget_bf_H0_I nontarget_bf_H0_D n_subj_ceiling_I n_subj_ceiling_D 

target_bf_H0 = np.asarray(target_bf_H0)[0]
nontarget_bf_H0_I = np.asarray(nontarget_bf_H0_I)[0]
nontarget_bf_H0_D = np.asarray(nontarget_bf_H0_D)[0]
n_subj_ceiling_I = np.asarray(n_subj_ceiling_I)[0]
n_subj_ceiling_D = np.asarray(n_subj_ceiling_D)[0]

# Prints results of calculations
print('Evidence of floor performance (immediate group): %.3g'% (nontarget_bf_H0_D))
print('Evidence of floor performance (delay group): %.3g'% (nontarget_bf_H0_I))
print('Percent of participants with ceiling performance (immediate group): ' + \
              '%d' % (100*n_subj_ceiling_I/batch_n_subjects))
print('Percent of participants with ceiling performance (delay group): ' + \
              '%d' % (100*n_subj_ceiling_D/batch_n_subjects))
print('Evidence of no difference between surprising and neutral targets (collapsed across groups): ' \
              + '%.3g' % (target_bf_H0))

# Stopping criterion calculations
Calculates stopping criteria tests for data from all batches up to current batch number to see if the criterion has been reached. Calculations from the 'Sample size determination' section of the manuscript.

## Stopping criterion - Bayes Factor calculations (R)
Calculates the Bayes Factor in favour of the alternative and in favour of the null for each of the following criteria:
- Main effect of surprise
- Interaction between question type (preS/preT) and surprise (S/N)
- 3-way interaction of question type, surprise and group (immediate/delay)

In [ ]:
%%R -i full_subj_avg_df

# Sets seed to arbitrary value at the beginning of each R cell, for reproducibility
set.seed(26)

# Takes only preS/preT questions for the stopping criteria analysis and removes all excluded subjects
retroactive_data <- subset(full_subj_avg_df, (q_type=='preS' | q_type=='preT') & exc_subj=='FALSE')

# Turns the subj column into a factor - will be converted to numeric by default
# Also sets the others to be factors (though they're already strings), since something
# in the conversion in the notebook works differently and at least one isn't recognised 
# as a factor otherwise
retroactive_data[,'subj'] <- as.factor(retroactive_data[,'subj'])
retroactive_data[,'q_type'] <- as.factor(retroactive_data[,'q_type'])
retroactive_data[,'surprise_type'] <- as.factor(retroactive_data[,'surprise_type'])
retroactive_data[,'group'] <- as.factor(retroactive_data[,'group'])

# Runs a 3-way anova with question-type, surprise and group
retro_bf = anovaBF(Pr ~ q_type*surprise_type*group + subj, data=retroactive_data, whichRandom='subj')

# Extracts BF in favour of the alternative (H1) and the null (H0) for each of the criteria
bf_main_surprise_H1 = as.numeric(as.vector(retro_bf[9]/retro_bf[4]))
bf_surprise_q_type_H1 = as.numeric(as.vector(retro_bf[17]/retro_bf[12]))
bf_3way_interaction_H1 = as.numeric(as.vector(retro_bf[18]/retro_bf[17]))
bf_main_surprise_H0 = 1/bf_main_surprise_H1
bf_surprise_q_type_H0 = 1/bf_surprise_q_type_H1
bf_3way_interaction_H0 = 1/bf_3way_interaction_H1

# Makes sure the correct models were taken from the full anova (in case the order can change)
models_validated <- TRUE
models_validated <- models_validated & setequal(unlist(strsplit(names(as.vector(retro_bf[4])), ' + ', TRUE)), 
                        c('group','q_type','group:q_type','subj'))
models_validated <- models_validated & setequal(unlist(strsplit(names(as.vector(retro_bf[9])), ' + ', TRUE)), 
                        c('group','q_type','group:q_type', 'surprise_type','subj'))
models_validated <- models_validated & setequal(unlist(strsplit(names(as.vector(retro_bf[12])), ' + ', TRUE)),
                        c('group','q_type','surprise_type', 'group:q_type','group:surprise_type','subj'))
models_validated <- models_validated & setequal(unlist(strsplit(names(as.vector(retro_bf[17])), ' + ', TRUE)),
                        c('group','q_type','surprise_type', 'group:q_type','group:surprise_type',
                          'q_type:surprise_type','subj'))
models_validated <- models_validated & setequal(unlist(strsplit(names(as.vector(retro_bf[18])), ' + ', TRUE)),
                        c('group','q_type','surprise_type', 'group:q_type','group:surprise_type',
                          'q_type:surprise_type','group:q_type:surprise_type', 'subj'))



## Stopping criterion - display results

In [ ]:
# Pulls the relevant variables from the R code and converts to python native data types

%Rpull models_validated bf_main_surprise_H1 bf_surprise_q_type_H1 bf_3way_interaction_H1 bf_main_surprise_H0 \
    bf_surprise_q_type_H0 bf_3way_interaction_H0

models_validated = bool(np.asarray(models_validated)[0])
bf_main_surprise_H1 = np.asarray(bf_main_surprise_H1)[0]
bf_surprise_q_type_H1 = np.asarray(bf_surprise_q_type_H1)[0]
bf_3way_interaction_H1 = np.asarray(bf_3way_interaction_H1)[0]
bf_main_surprise_H0 = np.asarray(bf_main_surprise_H0)[0]
bf_surprise_q_type_H0 = np.asarray(bf_surprise_q_type_H0)[0]
bf_3way_interaction_H0 = np.asarray(bf_3way_interaction_H0)[0]

# Determines whether stopping criterion has been reached. Either H1 in one of the tests or
# H0 in all three tests
if ((bf_main_surprise_H1 >= bf_thresh) or (bf_surprise_q_type_H1 >= bf_thresh) or \
    (bf_3way_interaction_H1 >= bf_thresh) or ((bf_main_surprise_H0 >= bf_thresh) and \
    (bf_surprise_q_type_H0 >= bf_thresh) and (bf_3way_interaction_H0 >= bf_thresh))):
    stop_criterion_reached = True
else:
    stop_criterion_reached = False


# Prints results of calculations, first verifying the chosen models are the correct ones
if (not models_validated):
    print('The ANOVA models chosen for the calculations do not match the expected ones.' + \
         'This may be due to different platforms/versions of the BayesFactor package.' + \
          'View cell above to identify the correct models from the bf variable (that includes)' + \
          'all models in the anova')
else:
    print('Evidence in favour of main effect of surprise (H1): %.3g' % (bf_main_surprise_H1))
    print('Evidence against main effect of surprise (H0): %.3g'% (bf_main_surprise_H0))
    print('Evidence in favour of q-type/surprise interaction (H1): %.3g'% (bf_surprise_q_type_H1))
    print('Evidence against q-type/surprise interaction (H0): %.3g'% (bf_surprise_q_type_H0))
    print('Evidence in favour of 3-way interaction (H1): %.3g'% (bf_3way_interaction_H1))
    print('Evidence against 3-way interaction (H0): %.3g'% (bf_3way_interaction_H0))
    
    if (stop_criterion_reached):
        print('Stop experiment - stopping criterion reached')
    elif (batch_num==max_n_batch):
        print('Stop experiment - stopping criterion not yet reached but max batches has')
    else:
        print('Continue experiment - stopping criterion not yet reached')

# Memory performance
Calculates average memory performance in each condition for reporting. Calculates both hit rate and Pr. Pr results are presented in Table 1 of the main text and hit rates in Supplementary Table 2

In [ ]:
# Calculates hit rates
full_subj_avg_df['hit_rate'] = full_subj_avg_df['n_hit']/(full_subj_avg_df['n_hit']+full_subj_avg_df['n_miss'])
full_subj_avg_df['HC_hit_rate'] = full_subj_avg_df['n_HC_hit']/(full_subj_avg_df['n_HC_hit']+\
        full_subj_avg_df['n_HC_miss'])

# Calculates overall mean performance (and standard error) for target questions, divided by group and surprise
T_mean = full_subj_avg_df[(~full_subj_avg_df['exc_subj'])&(full_subj_avg_df['q_type']=='T')]\
                .groupby(['group', 'surprise_type'])['hit_rate', 'HC_hit_rate', 'Pr', 'HC_Pr']\
                .mean().reset_index()   
T_sem = full_subj_avg_df[(~full_subj_avg_df['exc_subj'])&(full_subj_avg_df['q_type']=='T')]\
                .groupby(['group', 'surprise_type'])['hit_rate', 'HC_hit_rate', 'Pr', 'HC_Pr']\
                .sem().reset_index()  

# Calculates overall mean performance (and standard error) over non-target questions, divided only by group
nonT_mean = full_subj_avg_df[(~full_subj_avg_df['exc_subj'])&(full_subj_avg_df['q_type']!='T')]\
                .groupby(['group'])['hit_rate', 'HC_hit_rate', 'Pr', 'HC_Pr'].mean().reset_index()   
nonT_sem = full_subj_avg_df[(~full_subj_avg_df['exc_subj'])&(full_subj_avg_df['q_type']!='T')]\
                .groupby(['group'])['hit_rate', 'HC_hit_rate', 'Pr', 'HC_Pr'].sem().reset_index()  

# Calculates mean and standard error for each action type X surprise X group ()
performance_mean = full_subj_avg_df[~full_subj_avg_df['exc_subj']] \
                        .groupby(['group', 'q_type', 'surprise_type'])['hit_rate', 'HC_hit_rate', \
                        'Pr', 'HC_Pr'].mean().reset_index()
performance_sem = full_subj_avg_df[~full_subj_avg_df['exc_subj']] \
                        .groupby(['group', 'q_type', 'surprise_type'])['hit_rate', 'HC_hit_rate', \
                        'Pr', 'HC_Pr'].sem().reset_index()
                        

print('mean performance - target questions:')
print(T_mean)
print('\nperformance sem - target questions:')
print(T_sem)
print('\nmean performance - all non-target questions:')
print(nonT_mean)
print('\nperformance sem - all non-target questions:')
print(nonT_sem)
print('\nmean performance - q-typeXsurpriseXgroup:')
print(performance_mean)
print('\nperformance sem - q-typeXsurpriseXgroup:')
print(performance_sem)

# Primary analyses

The analyses described in the 'Retroactive and proactive effects of surprise' section of the manuscript.

## Proactive ANOVA (R)

Runs a 3-way ANOVA for proactive effects (the retroactive ANOVA has already been calculated for the stopping criterion)

In [ ]:
%%R -i full_subj_avg_df

# Sets seed to arbitrary value at the beginning of each R cell, for reproducibility
set.seed(26)

# Turns the subj column into a factor - will be converted to numeric by default
# Also sets the others to be factors (though they're already strings), since something
# in the conversion in the notebook works differently and at least one isn't recognised 
# as a factor otherwise
full_subj_avg_df[,'subj'] <- as.factor(full_subj_avg_df[,'subj'])
full_subj_avg_df[,'q_type'] <- as.factor(full_subj_avg_df[,'q_type'])
full_subj_avg_df[,'surprise_type'] <- as.factor(full_subj_avg_df[,'surprise_type'])
full_subj_avg_df[,'group'] <- as.factor(full_subj_avg_df[,'group'])

# Gets the subset of data for the proactive test
proactive_data <- subset(full_subj_avg_df, (q_type=='preT' | q_type=='postT') & exc_subj=='FALSE')

# Runs a 3-way anova for the proactive effects, and all subsequent comparisons that were run for the 
# retroactive effects
pro_bf = anovaBF(Pr ~ q_type*surprise_type*group + subj, data=proactive_data, 
                 whichRandom='subj')

# Extracts BF in favour of the alternative (H1) for each of the tests described above
pro_bf_main_surprise_H1 = as.numeric(as.vector(pro_bf[9]/pro_bf[4]))
pro_bf_surprise_q_type_H1 = as.numeric(as.vector(pro_bf[17]/pro_bf[12]))
pro_bf_3way_interaction_H1 = as.numeric(as.vector(pro_bf[18]/pro_bf[17]))

# Makes sure the correct models were taken from the full anova (in case the order can change)
pro_validated <- TRUE
pro_validated <- pro_validated & setequal(unlist(strsplit(names(as.vector(pro_bf[4])), ' + ', TRUE)), 
                        c('group','q_type','group:q_type','subj'))
pro_validated <- pro_validated & setequal(unlist(strsplit(names(as.vector(pro_bf[9])), ' + ', TRUE)), 
                        c('group','q_type','group:q_type', 'surprise_type','subj'))
pro_validated <- pro_validated & setequal(unlist(strsplit(names(as.vector(pro_bf[12])), ' + ', TRUE)),
                        c('group','q_type','surprise_type', 'group:q_type','group:surprise_type','subj'))
pro_validated <- pro_validated & setequal(unlist(strsplit(names(as.vector(pro_bf[17])), ' + ', TRUE)),
                        c('group','q_type','surprise_type', 'group:q_type','group:surprise_type',
                          'q_type:surprise_type','subj'))
pro_validated <- pro_validated & setequal(unlist(strsplit(names(as.vector(pro_bf[18])), ' + ', TRUE)),
                        c('group','q_type','surprise_type', 'group:q_type','group:surprise_type',
                          'q_type:surprise_type','group:q_type:surprise_type', 'subj'))


## Retroactive and proactive planned t-tests (R)

Runs t-tests for the effect of surprise (Pr Surprise - Pr Neutral) for each question-type:
- within each group
- collapsing across groups
- direct comparison between the groups

In [ ]:
%%R -i full_diff_df

# Sets seed to arbitrary value at the beginning of each R cell, for reproducibility
set.seed(26)

# Turns the subj column into a factor - will be converted to numeric by default
# Also sets the others to be factors (though they're already strings), since something
# in the conversion in the notebook works differently and at least one isn't recognised 
# as a factor otherwise
full_diff_df[,'subj'] <- as.factor(full_diff_df[,'subj'])
full_diff_df[,'q_type'] <- as.factor(full_diff_df[,'q_type'])
full_diff_df[,'group'] <- as.factor(full_diff_df[,'group'])

# Slices the data to have the relevant subsets for each test
preS_I_diff <- subset(full_diff_df, q_type=='preS' & group=='I' & exc_subj=='FALSE')
preT_I_diff <- subset(full_diff_df, q_type=='preT' & group=='I' & exc_subj=='FALSE')
postT_I_diff <- subset(full_diff_df, q_type=='postT' & group=='I' & exc_subj=='FALSE')
preS_D_diff <- subset(full_diff_df, q_type=='preS' & group=='D' & exc_subj=='FALSE')
preT_D_diff <- subset(full_diff_df, q_type=='preT' & group=='D' & exc_subj=='FALSE')
postT_D_diff <- subset(full_diff_df, q_type=='postT' & group=='D' & exc_subj=='FALSE')
preS_comb_diff <- subset(full_diff_df, q_type=='preS' & exc_subj=='FALSE')
preT_comb_diff <- subset(full_diff_df, q_type=='preT' & exc_subj=='FALSE')
postT_comb_diff <- subset(full_diff_df, q_type=='postT' & exc_subj=='FALSE')


# Runs the planned t-tests, looking at surprise effects within each question type
preS_I_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preS_I_diff['Pr_diff'])))))
preT_I_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preT_I_diff['Pr_diff'])))))
postT_I_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(postT_I_diff['Pr_diff'])))))
preS_D_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preS_D_diff['Pr_diff'])))))
preT_D_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preT_D_diff['Pr_diff'])))))
postT_D_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(postT_D_diff['Pr_diff'])))))
preS_comb_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preS_comb_diff['Pr_diff'])))))
preT_comb_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preT_comb_diff['Pr_diff'])))))
postT_comb_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(postT_comb_diff['Pr_diff'])))))
preS_comp_bf_H1 <- as.numeric(as.vector(ttestBF(x=as.numeric(unlist(preS_I_diff['Pr_diff'])), 
                                                  y=as.numeric(unlist(preS_D_diff['Pr_diff'])))))
preT_comp_bf_H1 <- as.numeric(as.vector(ttestBF(x=as.numeric(unlist(preT_I_diff['Pr_diff'])), 
                                                  y=as.numeric(unlist(preT_D_diff['Pr_diff'])))))
postT_comp_bf_H1 <- as.numeric(as.vector(ttestBF(x=as.numeric(unlist(postT_I_diff['Pr_diff'])), 
                                                  y=as.numeric(unlist(postT_D_diff['Pr_diff'])))))

## Primary analyses - displays results
Display evidence in favour of H1 (BF10) / in favour of the null (BF01) for each test

In [ ]:
# Pulls the relevant variables from the R code and converts to python native data types

%Rpull pro_validated pro_bf_main_surprise_H1 pro_bf_surprise_q_type_H1 pro_bf_3way_interaction_H1 \
        preS_I_bf_H1 preT_I_bf_H1 postT_I_bf_H1 preS_D_bf_H1 preT_D_bf_H1 postT_D_bf_H1 preS_comb_bf_H1 \
        preT_comb_bf_H1 postT_comb_bf_H1 preS_comp_bf_H1 preT_comp_bf_H1 postT_comp_bf_H1

pro_validated = bool(np.asarray(pro_validated)[0])
pro_bf_main_surprise_H1 = np.asarray(pro_bf_main_surprise_H1)[0]
pro_bf_surprise_q_type_H1 = np.asarray(pro_bf_surprise_q_type_H1)[0]
pro_bf_3way_interaction_H1 = np.asarray(pro_bf_3way_interaction_H1)[0]
preS_I_bf_H1 = np.asarray(preS_I_bf_H1)[0]
preT_I_bf_H1 = np.asarray(preT_I_bf_H1)[0]
postT_I_bf_H1 = np.asarray(postT_I_bf_H1)[0]
preS_D_bf_H1 = np.asarray(preS_D_bf_H1)[0]
preT_D_bf_H1 = np.asarray(preT_D_bf_H1)[0]
postT_D_bf_H1 = np.asarray(postT_D_bf_H1)[0]
preS_comb_bf_H1 = np.asarray(preS_comb_bf_H1)[0]
preT_comb_bf_H1 = np.asarray(preT_comb_bf_H1)[0]
postT_comb_bf_H1 = np.asarray(postT_comb_bf_H1)[0]
preS_comp_bf_H1 = np.asarray(preS_comp_bf_H1)[0]
preT_comp_bf_H1 = np.asarray(preT_comp_bf_H1)[0]
postT_comp_bf_H1 = np.asarray(postT_comp_bf_H1)[0]


# Prints results of calculations, first verifying the chosen models are the correct ones
if (not pro_validated):
    print('The ANOVA models chosen for the calculations do not match the expected ones.' + \
         'This may be due to different platforms/versions of the BayesFactor package.' + \
          'View cell above to identify the correct models from the bf variable (that includes)' + \
          'all models in the anova')
else:
    print ('Results of proactive ANOVA:')
    print('   Evidence for/against main effect of surprise: %.3g/%.3g'% \
          (pro_bf_main_surprise_H1,1/pro_bf_main_surprise_H1))
    print('   Evidence for/against q-type/surprise interaction: %.3g/%.3g'% \
          (pro_bf_surprise_q_type_H1, 1/pro_bf_surprise_q_type_H1))
    print('   Evidence for/against 3-way interaction: %.3g/%.3g'% \
          (pro_bf_3way_interaction_H1, 1/pro_bf_3way_interaction_H1))
    print('\nResults of planned t-tests:')
    print('   Evidence for/against surprise effect in preS (immediate group): '\
              + '%.3g/%.3g'% (preS_I_bf_H1, 1/preS_I_bf_H1))
    print('   Evidence for/against surprise effect in preT (immediate group): '\
              + '%.3g/%.3g'% (preT_I_bf_H1, 1/preT_I_bf_H1))
    print('   Evidence for/against surprise effect in postT (immediate group): '\
              + '%.3g/%.3g'% (postT_I_bf_H1, 1/postT_I_bf_H1))
    print('   Evidence for/against surprise effect in preS (delay group): '\
              + '%.3g/%.3g'% (preS_D_bf_H1, 1/preS_D_bf_H1))
    print('   Evidence for/against surprise effect in preT (delay group): '\
              + '%.3g/%.3g'% (preT_D_bf_H1, 1/preT_D_bf_H1))
    print('   Evidence for/against surprise effect in postT (delay group): '\
              + '%.3g/%.3g'% (postT_D_bf_H1, 1/postT_D_bf_H1))
    print('   Evidence for/against surprise effect in preS (collapsed): ' + \
              '%.3g/%.3g'% (preS_comb_bf_H1, 1/preS_comb_bf_H1))
    print('   Evidence for/against surprise effect in preT (collapsed): ' + \
              '%.3g/%.3g'% (preT_comb_bf_H1, 1/preT_comb_bf_H1))
    print('   Evidence for/against surprise effect in postT (collapsed): ' + \
              '%.3g/%.3g'% (postT_comb_bf_H1, 1/postT_comb_bf_H1))
    print('   Evidence for/against surprise effect in preS (comparison between groups): ' + \
              '%.3g/%.3g'% (preS_comp_bf_H1, 1/preS_comp_bf_H1))
    print('   Evidence for/against surprise effect in preT (comparison between groups): ' + \
              '%.3g/%.3g'% (preT_comp_bf_H1, 1/preT_comp_bf_H1))
    print('   Evidence for/against surprise effect in postT (comparison between groups): ' + \
              '%.3g/%.3g'% (postT_comp_bf_H1, 1/postT_comp_bf_H1))
    

# Plot results

Plots the surprise effect per question type, separately for each group, so the results of the analyses can be interpreted. Also creates and displays a dataframe with the mean, standard deviation and standard error for each question type X group. Figure 2 of main text.

In [ ]:
# Plots the surprise effect for each group, as well as combined across groups

for group in (['immediate', 'delay']):
    fig, ax = plt.subplots(figsize=(fig_w, fig_h), dpi=plot_dpi)
    ax.axhline(0, color='k')
    sns.violinplot(x='q_type', y='Pr_diff',data=full_diff_df[(full_diff_df['group']==group[0].upper())&\
                        (full_diff_df['exc_subj']==False)], linewidth=2.5, ax=ax, 
                           order=['preS', 'preT', 'T', 'postT'], palette='Set2', saturation=1)
    fig.suptitle(group.capitalize(), fontsize=title_font)
    plt.xlabel('Action Type', fontsize=label_font)
    plt.ylabel('Surprise effect (Pr-S - Pr-N)', fontsize=label_font)
    plt.yticks(np.arange(-1,1,0.5), fontsize=tick_font)
    plt.xticks(fontsize=tick_font)
    if (save_figs):
        fig.savefig(f'{fig_dir}surprise_effect_{group}.eps')

fig, ax = plt.subplots(figsize=(fig_w, fig_h), dpi=plot_dpi)
ax.axhline(0, color='k')
sns.violinplot(x='q_type', y='Pr_diff',data=full_diff_df[(full_diff_df['exc_subj']==False)],
               linewidth=2.5, ax=ax, order=['preS', 'preT', 'T', 'postT'], palette='Set2', saturation=1)
fig.suptitle('Collapsed', fontsize=title_font)
plt.xlabel('Action Type', fontsize=label_font)
plt.ylabel('Surprise effect (Pr-S - Pr-N)', fontsize=label_font)
plt.yticks(np.arange(-1,1,0.5), fontsize=tick_font)
plt.xticks(fontsize=tick_font)
if (save_figs):
    fig.savefig(f'{fig_dir}surprise_effect_collapsed.eps')


#### Plots for Supplementary Figure 3 - difference in surprise effect between action types

In [ ]:
# Plots the difference in the surprise effect (preT-preS and postT-preT) for each group, as well as combined across groups

# First creates a version of the dataframe with the preT-preS and postT-preT surprise differences
effect_diff_df = full_diff_df.pivot_table(index=['subj', 'group', 'exc_subj'], 
                                           columns='q_type', values=['Pr_diff', 'HC_Pr_diff'])

effect_diff_df.columns = list(map("_".join, effect_diff_df.columns))
effect_diff_df.reset_index(inplace=True)
effect_diff_df['preT_preS_Pr_diff'] = effect_diff_df['Pr_diff_preT']-effect_diff_df['Pr_diff_preS']
effect_diff_df['preT_preS_HC_Pr_diff'] = effect_diff_df['HC_Pr_diff_preT']-effect_diff_df['HC_Pr_diff_preS']
effect_diff_df['postT_preT_Pr_diff'] = effect_diff_df['Pr_diff_postT']-effect_diff_df['Pr_diff_preT']
effect_diff_df['postT_preT_HC_Pr_diff'] = effect_diff_df['HC_Pr_diff_postT']-effect_diff_df['HC_Pr_diff_preT']

# Plots the preT-preS difference alongside the postT-preT difference
for group in (['immediate', 'delay']):
    fig, ax = plt.subplots(figsize=(fig_w,fig_h), dpi=plot_dpi)
    ax.axhline(0, color='k')
    sns.violinplot(data=effect_diff_df[(effect_diff_df['group']==group[0].upper())&\
                        (effect_diff_df['exc_subj']==False)][['preT_preS_Pr_diff', 'postT_preT_Pr_diff']], linewidth=2.5, ax=ax, 
                           order=['preT_preS_Pr_diff', 'postT_preT_Pr_diff'], palette=[(0.16, 0.52, 0.77),(0.88, 0.51, 0.17)], saturation=1)
    fig.suptitle(group.capitalize(), fontsize=title_font)
    plt.xlabel('Action Type', fontsize=label_font)
    plt.ylabel('Surprise effect difference', fontsize=label_font)
    plt.yticks(np.arange(-1,1,0.5), fontsize=tick_font)
    plt.xticks(fontsize=tick_font)
    if (save_figs):
        fig.savefig(f'{fig_dir}surprise_effect_diff_{group}.eps')

fig, ax = plt.subplots(figsize=(fig_w,fig_h), dpi=plot_dpi)
ax.axhline(0, color='k')
sns.violinplot(data=effect_diff_df[(effect_diff_df['exc_subj']==False)][['preT_preS_Pr_diff', 'postT_preT_Pr_diff']],
               linewidth=2.5, ax=ax, order=['preT_preS_Pr_diff', 'postT_preT_Pr_diff'], palette=[(0.16, 0.52, 0.77),(0.88, 0.51, 0.17)], saturation=1)
fig.suptitle('Collapsed', fontsize=title_font)
plt.xlabel('Question Type', fontsize=label_font)
plt.ylabel('Action effect difference', fontsize=label_font)
plt.yticks(np.arange(-1,1,0.5), fontsize=tick_font)
plt.xticks(fontsize=tick_font)
if (save_figs):
    fig.savefig(f'{fig_dir}surprise_effect_diff_collapsed.eps')


#### Displays mean, sd, and se in each condition

In [ ]:
pd.options.display.float_format = '{:.3g}'.format

# Creates dataframe wth mean, sd and se of each condition and displays
mean_effect_df = full_diff_df[(full_diff_df['exc_subj']==False)].groupby(['q_type', 'group'])['Pr_diff'].\
                    agg(['mean', 'std', 'sem'])
display(mean_effect_df)

# Creates dataframe wth mean, sd and se of preT-preS and postT-preT differences, and displays

mean_diff_effect_df = effect_diff_df[(effect_diff_df['exc_subj']==False)].groupby(['group'])[['preT_preS_Pr_diff', 'preT_preS_HC_Pr_diff', 'postT_preT_Pr_diff',
                            'postT_preT_HC_Pr_diff']].agg(['mean', 'std', 'sem'])

display(mean_diff_effect_df)

#### Calculates effect size

Calculates the effect size for the postT effects - both for postT itself and for the difference between preT and postT (the interaction). Does this once for the delay group and once for both groups combined

In [ ]:
# Calculates effect size for the postT effects

# postT in delay group
postT_delay_subset = full_diff_df[(full_diff_df['exc_subj']==False)&(full_diff_df['q_type']=='postT')&\
                                   (full_diff_df['group']=='D')]
eff_size_postT_delay = np.mean(postT_delay_subset['Pr_diff'])/np.std(postT_delay_subset['Pr_diff'])

# postT collapsed across groups
postT_comb_subset = full_diff_df[(full_diff_df['exc_subj']==False)&(full_diff_df['q_type']=='postT')]
eff_size_postT_comb = np.mean(postT_comb_subset['Pr_diff'])/np.std(postT_comb_subset['Pr_diff'])

# Creates a version of the dataframe with the surprise effect (Pr-S - Pr-N) of postT subtracted from
# the effect of preT to estimate the effect size of the difference
preT_diff_df = full_diff_df[full_diff_df['q_type']=='preT'].drop(\
                columns=['Pr-S', 'HC_Pr-S', 'Pr-N', 'HC_Pr-N', 'q_type']).\
                rename({'Pr_diff': 'Pr_diff_preT', 'HC_Pr_diff': 'HC_Pr_diff_preT'}, axis=1)
postT_diff_df = full_diff_df[full_diff_df['q_type']=='postT'].drop(\
                columns=['Pr-S', 'HC_Pr-S', 'Pr-N', 'HC_Pr-N', 'q_type']).\
                rename({'Pr_diff': 'Pr_diff_postT', 'HC_Pr_diff': 'HC_Pr_diff_postT'}, axis=1)
pro_diff_df = preT_diff_df.merge(postT_diff_df, validate='one_to_one')
pro_diff_df['Pr_diff'] = pro_diff_df['Pr_diff_preT']-pro_diff_df['HC_Pr_diff_postT']
pro_diff_df['HC_Pr_diff'] = pro_diff_df['HC_Pr_diff_preT']-pro_diff_df['HC_Pr_diff_postT']

# preT vs. postT in delay group
pro_diff_delay = pro_diff_df[(pro_diff_df['exc_subj']==False)&(pro_diff_df['group']=='D')]
eff_size_pro_diff_delay = np.mean(pro_diff_delay['Pr_diff'])/np.std(pro_diff_delay['Pr_diff'])

# preT vs. postT collapsed across groups
pro_diff_comb = pro_diff_df[(pro_diff_df['exc_subj']==False)]
eff_size_pro_diff_comb = np.mean(pro_diff_comb['Pr_diff'])/np.std(pro_diff_comb['Pr_diff'])

print('Proactive effect sizes:')
print('   postT effect (t-test) - delay group: %.3g'% eff_size_postT_delay)
print('   postT effect (t-test) - combined: %.3g'% eff_size_postT_comb)
print('   preT-postT diff (interaction) - delay group: %.3g'% eff_size_pro_diff_delay)
print('   preT-postT diff (interaction) - combined: %.3g'% eff_size_pro_diff_comb)

# Secondary analyses - high-confidence Pr

Runs all of the analyses a second time, looking at high-confidence Pr (Hit-FA for answers with confidence of 2) instead of regular Pr (confidence of 1/2)

## Secondary analyses - HC Pr - retroactive and proactive ANOVAs (R)

Same as the retroactive and proactive analyses above. Skips model validation since model order should be identical regardless of the dependent variable

In [ ]:
%%R -i full_subj_avg_df

# Sets seed to arbitrary value at the beginning of each R cell, for reproducibility
set.seed(26)

# Turns the subj column into a factor - will be converted to numeric by default
# Also sets the others to be factors (though they're already strings), since something
# in the conversion in the notebook works differently and at least one isn't recognised 
# as a factor otherwise
full_subj_avg_df[,'subj'] <- as.factor(full_subj_avg_df[,'subj'])
full_subj_avg_df[,'q_type'] <- as.factor(full_subj_avg_df[,'q_type'])
full_subj_avg_df[,'surprise_type'] <- as.factor(full_subj_avg_df[,'surprise_type'])
full_subj_avg_df[,'group'] <- as.factor(full_subj_avg_df[,'group'])

# Takes only preS/preT questions for retroactive analysis, preT/postT for proactive, 
# and removes all excluded subjects in both. These will be identical to those calculated above,
# but duplicates code so each analysis can be run independently
retroactive_data <- subset(full_subj_avg_df, (q_type=='preS' | q_type=='preT') & exc_subj=='FALSE')
proactive_data <- subset(full_subj_avg_df, (q_type=='preT' | q_type=='postT') & exc_subj=='FALSE')

# Runs 3-way anovas with question-type, surprise and group, here using HC Pr
retro_HC_bf = anovaBF(HC_Pr ~ q_type*surprise_type*group + subj, 
                      data=retroactive_data, whichRandom='subj')
pro_HC_bf = anovaBF(HC_Pr ~ q_type*surprise_type*group + subj, 
                    data=proactive_data, whichRandom='subj')

# Extracts BF in favour of the alternative (H1) for each of the criteria
retro_HC_bf_main_surprise_H1 = as.numeric(as.vector(retro_HC_bf[9]/retro_HC_bf[4]))
retro_HC_bf_surprise_q_type_H1 = as.numeric(as.vector(retro_HC_bf[17]/retro_HC_bf[12]))
retro_HC_bf_3way_interaction_H1 = as.numeric(as.vector(retro_HC_bf[18]/retro_HC_bf[17]))
pro_HC_bf_main_surprise_H1 = as.numeric(as.vector(pro_HC_bf[9]/pro_HC_bf[4]))
pro_HC_bf_surprise_q_type_H1 = as.numeric(as.vector(pro_HC_bf[17]/pro_HC_bf[12]))
pro_HC_bf_3way_interaction_H1 = as.numeric(as.vector(pro_HC_bf[18]/pro_HC_bf[17])) 

## Secondary analyses - HC Pr - retroactive and proactive ANOVAs - display results

Displays results of the high-confidence anovas

In [ ]:
# Pulls the relevant variables from the R code and converts to python native data types

%Rpull retro_HC_bf_main_surprise_H1 retro_HC_bf_surprise_q_type_H1 retro_HC_bf_3way_interaction_H1 \
pro_HC_bf_main_surprise_H1 pro_HC_bf_surprise_q_type_H1 pro_HC_bf_3way_interaction_H1

retro_HC_bf_main_surprise_H1 = np.asarray(retro_HC_bf_main_surprise_H1)[0]
retro_HC_bf_surprise_q_type_H1 = np.asarray(retro_HC_bf_surprise_q_type_H1)[0]
retro_HC_bf_3way_interaction_H1 = np.asarray(retro_HC_bf_3way_interaction_H1)[0]
pro_HC_bf_main_surprise_H1 = np.asarray(pro_HC_bf_main_surprise_H1)[0]
pro_HC_bf_surprise_q_type_H1 = np.asarray(pro_HC_bf_surprise_q_type_H1)[0]
pro_HC_bf_3way_interaction_H1 = np.asarray(pro_HC_bf_3way_interaction_H1)[0]
retro_HC_bf_main_surprise_H0 = 1/retro_HC_bf_main_surprise_H1
retro_HC_bf_surprise_q_type_H0 = 1/retro_HC_bf_surprise_q_type_H1
retro_HC_bf_3way_interaction_H0 = 1/retro_HC_bf_3way_interaction_H1
pro_HC_bf_main_surprise_H0 = 1/pro_HC_bf_main_surprise_H1
pro_HC_bf_surprise_q_type_H0 = 1/pro_HC_bf_surprise_q_type_H1
pro_HC_bf_3way_interaction_H0 = 1/pro_HC_bf_3way_interaction_H1


print ('Results of retroactive HC-Pr ANOVA:')
print('   Evidence in favour of main effect of surprise (H1): %.3g'% retro_HC_bf_main_surprise_H1)
print('   Evidence against main effect of surprise (H0): %.3g'% retro_HC_bf_main_surprise_H0)
print('   Evidence in favour of q-type/surprise interaction (H1): %.3g'% (retro_HC_bf_surprise_q_type_H1))
print('   Evidence against q-type/surprise interaction (H0): %.3g'% (retro_HC_bf_surprise_q_type_H0))
print('   Evidence in favour of 3-way interaction (H1): %.3g'% (retro_HC_bf_3way_interaction_H1))
print('   Evidence against 3-way interaction (H0): %.3g'% (retro_HC_bf_3way_interaction_H0))

print ('\nResults of proactive HC-Pr ANOVA:')
print('   Evidence in favour of main effect of surprise (H1): %.3g'% pro_HC_bf_main_surprise_H1)
print('   Evidence against main effect of surprise (H0): %.3g'% pro_HC_bf_main_surprise_H0)
print('   Evidence in favour of q-type/surprise interaction (H1): %.3g'% (pro_HC_bf_surprise_q_type_H1))
print('   Evidence against q-type/surprise interaction (H0): %.3g'% (pro_HC_bf_surprise_q_type_H0))
print('   Evidence in favour of 3-way interaction (H1): %.3g'% (pro_HC_bf_3way_interaction_H1))
print('   Evidence against 3-way interaction (H0): %.3g'% (pro_HC_bf_3way_interaction_H0))

## Secondary analyses - HC Pr - planned t-tests (R)

Same as the retroactive and proactive planned t-tests above

In [ ]:
%%R -i full_diff_df

# Sets seed to arbitrary value at the beginning of each R cell, for reproducibility
set.seed(26)

# Turns the subj column into a factor - will be converted to numeric by default
# Also sets the others to be factors (though they're already strings), since something
# in the conversion in the notebook works differently and at least one isn't recognised 
# as a factor otherwise
full_diff_df[,'subj'] <- as.factor(full_diff_df[,'subj'])
full_diff_df[,'q_type'] <- as.factor(full_diff_df[,'q_type'])
full_diff_df[,'group'] <- as.factor(full_diff_df[,'group'])

# Slices the data to have the relevant subsets for each test
preS_I_diff <- subset(full_diff_df, q_type=='preS' & group=='I' & exc_subj=='FALSE')
preT_I_diff <- subset(full_diff_df, q_type=='preT' & group=='I' & exc_subj=='FALSE')
postT_I_diff <- subset(full_diff_df, q_type=='postT' & group=='I' & exc_subj=='FALSE')
preS_D_diff <- subset(full_diff_df, q_type=='preS' & group=='D' & exc_subj=='FALSE')
preT_D_diff <- subset(full_diff_df, q_type=='preT' & group=='D' & exc_subj=='FALSE')
postT_D_diff <- subset(full_diff_df, q_type=='postT' & group=='D' & exc_subj=='FALSE')
preS_comb_diff <- subset(full_diff_df, q_type=='preS' & exc_subj=='FALSE')
preT_comb_diff <- subset(full_diff_df, q_type=='preT' & exc_subj=='FALSE')
postT_comb_diff <- subset(full_diff_df, q_type=='postT' & exc_subj=='FALSE')


# Runs the planned t-tests, looking at surprise effects within each question type. Here takes HC_Pr_diff
# instead of Pr_diff
HC_preS_I_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preS_I_diff['HC_Pr_diff'])))))
HC_preT_I_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preT_I_diff['HC_Pr_diff'])))))
HC_postT_I_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(postT_I_diff['HC_Pr_diff'])))))
HC_preS_D_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preS_D_diff['HC_Pr_diff'])))))
HC_preT_D_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preT_D_diff['HC_Pr_diff'])))))
HC_postT_D_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(postT_D_diff['HC_Pr_diff'])))))
HC_preS_comb_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preS_comb_diff['HC_Pr_diff'])))))
HC_preT_comb_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(preT_comb_diff['HC_Pr_diff'])))))
HC_postT_comb_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(postT_comb_diff['HC_Pr_diff'])))))
HC_preS_comp_bf_H1 <- as.numeric(as.vector(ttestBF(x=as.numeric(unlist(preS_I_diff['HC_Pr_diff'])), 
                                                  y=as.numeric(unlist(preS_D_diff['HC_Pr_diff'])))))
HC_preT_comp_bf_H1 <- as.numeric(as.vector(ttestBF(x=as.numeric(unlist(preT_I_diff['HC_Pr_diff'])), 
                                                  y=as.numeric(unlist(preT_D_diff['HC_Pr_diff'])))))
HC_postT_comp_bf_H1 <- as.numeric(as.vector(ttestBF(x=as.numeric(unlist(postT_I_diff['HC_Pr_diff'])), 
                                                  y=as.numeric(unlist(postT_D_diff['HC_Pr_diff'])))))

## Secondary analyses - HC Pr - planned t-tests - display results

Displays only evidence in favour of H1 (BF10) / in favour of the null (BF01) for each test 

In [ ]:
# Pulls the relevant variables from the R code and converts to python native data types

%Rpull HC_preS_I_bf_H1 HC_preT_I_bf_H1 HC_postT_I_bf_H1 HC_preS_D_bf_H1 HC_preT_D_bf_H1 HC_postT_D_bf_H1 \
HC_preS_comb_bf_H1 HC_preT_comb_bf_H1 HC_postT_comb_bf_H1 HC_preS_comp_bf_H1 HC_preT_comp_bf_H1 \
HC_postT_comp_bf_H1

HC_preS_I_bf_H1 = np.asarray(HC_preS_I_bf_H1)[0]
HC_preT_I_bf_H1 = np.asarray(HC_preT_I_bf_H1)[0]
HC_postT_I_bf_H1 = np.asarray(HC_postT_I_bf_H1)[0]
HC_preS_D_bf_H1 = np.asarray(HC_preS_D_bf_H1)[0]
HC_preT_D_bf_H1 = np.asarray(HC_preT_D_bf_H1)[0]
HC_postT_D_bf_H1 = np.asarray(HC_postT_D_bf_H1)[0]
HC_preS_comb_bf_H1 = np.asarray(HC_preS_comb_bf_H1)[0]
HC_preT_comb_bf_H1 = np.asarray(HC_preT_comb_bf_H1)[0]
HC_postT_comb_bf_H1 = np.asarray(HC_postT_comb_bf_H1)[0]
HC_preS_comp_bf_H1 = np.asarray(HC_preS_comp_bf_H1)[0]
HC_preT_comp_bf_H1 = np.asarray(HC_preT_comp_bf_H1)[0]
HC_postT_comp_bf_H1 = np.asarray(HC_postT_comp_bf_H1)[0]

print('Evidence for/against surprise effect in preS (immediate group): '\
          + '%.3g/%.3g'% (HC_preS_I_bf_H1, 1/HC_preS_I_bf_H1))
print('Evidence for/against surprise effect in preT (immediate group): '\
          + '%.3g/%.3g'% (HC_preT_I_bf_H1, 1/HC_preT_I_bf_H1))
print('Evidence for/against surprise effect in postT (immediate group): '\
          + '%.3g/%.3g'% (HC_postT_I_bf_H1, 1/HC_postT_I_bf_H1))
print('Evidence for/against surprise effect in preS (delay group): '\
          + '%.3g/%.3g'% (HC_preS_D_bf_H1, 1/HC_preS_D_bf_H1))
print('Evidence for/against surprise effect in preT (delay group): '\
          + '%.3g/%.3g'% (HC_preT_D_bf_H1, 1/HC_preT_D_bf_H1))
print('Evidence for/against surprise effect in postT (delay group): '\
          + '%.3g/%.3g'% (HC_postT_D_bf_H1, 1/HC_postT_D_bf_H1))
print('Evidence for/against surprise effect in preS (collapsed): ' + \
          '%.3g/%.3g'% (HC_preS_comb_bf_H1, 1/HC_preS_comb_bf_H1))
print('Evidence for/against surprise effect in preT (collapsed): ' + \
          '%.3g/%.3g'% (HC_preT_comb_bf_H1, 1/HC_preT_comb_bf_H1))
print('Evidence for/against surprise effect in postT (collapsed): ' + \
          '%.3g/%.3g'% (HC_postT_comb_bf_H1, 1/HC_postT_comb_bf_H1))
print('Evidence for/against surprise effect in preS (comparison between groups): ' + \
          '%.3g/%.3g'% (HC_preS_comp_bf_H1, 1/HC_preS_comp_bf_H1))
print('Evidence for/against surprise effect in preT (comparison between groups): ' + \
          '%.3g/%.3g'% (HC_preT_comp_bf_H1, 1/HC_preT_comp_bf_H1))
print('Evidence for/against surprise effect in postT (comparison between groups): ' + \
          '%.3g/%.3g'% (HC_postT_comp_bf_H1, 1/HC_postT_comp_bf_H1))

## Secondary analyses - HC Pr - Plot results

As above, plots the surprise effect per question type, separately for each group, so the results of the analyses can be interpreted. Also creates a dataframe with the mean, standard deviation and standard error for each question type X group. Here uses high-confidence Pr instead of Pr

In [ ]:
# Plots the surprise effect for each group, as well as combined across groups

for group in (['immediate', 'delay']):
    fig, ax = plt.subplots(figsize=(fig_w,fig_h), dpi=plot_dpi)
    ax.axhline(0, color='k')
    sns.violinplot(x='q_type', y='HC_Pr_diff',data=full_diff_df[(full_diff_df['group']==group[0].upper())&\
                        (full_diff_df['exc_subj']==False)], linewidth=2.5, ax=ax, 
                           order=['preS', 'preT', 'T', 'postT'], palette='Set2', saturation=1)
    fig.suptitle('Surprise effect - ' + group + ' group', fontsize=title_font)
    plt.xlabel('Question Type', fontsize=label_font)
    plt.ylabel('Surprise effect (Pr-S - Pr-N)', fontsize=label_font)
    plt.yticks(np.arange(-1,1,0.5), fontsize=tick_font)
    plt.xticks(fontsize=tick_font)
    if (save_figs):
        fig.savefig(f'{fig_dir}surprise_effect_HC_{group}.eps')

fig, ax = plt.subplots(figsize=(fig_w,fig_h), dpi=plot_dpi)
ax.axhline(0, color='k')
sns.violinplot(x='q_type', y='HC_Pr_diff',data=full_diff_df[(full_diff_df['exc_subj']==False)],
               linewidth=2.5, ax=ax, order=['preS', 'preT', 'T', 'postT'], palette='Set2', saturation=1)
fig.suptitle('Surprise effect - both groups', fontsize=title_font)
plt.xlabel('Question Type', fontsize=label_font)
plt.ylabel('Surprise effect (Pr-S - Pr-N)', fontsize=label_font)
plt.yticks(np.arange(-1,1,0.5), fontsize=tick_font)
plt.xticks(fontsize=tick_font)
if (save_figs):
    fig.savefig(f'{fig_dir}surprise_effect_HC_collapsed.eps')


In [ ]:
# Creates dataframe wth mean, sd and se of each condition and displays
HC_mean_effect_df = full_diff_df[(full_diff_df['exc_subj']==False)].groupby(['q_type', 'group'])\
                        ['HC_Pr_diff'].agg(['mean', 'std', 'sem'])
display(HC_mean_effect_df)

# Post-hoc control analysis (counterbalancing)

As the study was run online, multiple participants did the experiment in parallel. This resulted in the counterbalancing not being accurate, as some participants stopped in the middle of the experiment. Since this could lead to spurious results, we include an additional test to assess whether the proactive effect is driven by the inaccurate counterbalancing - a single trial analysis using brms, accounting for order and version

## Displays counterbalancing info

To assess the accuracy of counterbalancing, displays the number of participants that viewed the films in each of the possible orders (six) and each of the possible versions (four versions, with different divisions of target scenes to neutral/surprise). Also displays the range of the number of participants in the 24 combinations of order and version. 

In [ ]:
# Counts the number of participants in each combination of order X version X group
cb_info_full = full_subj_avg_df[~full_subj_avg_df['exc_subj']][['subj','order_idx','mov_ver','group']].drop_duplicates().groupby(\
                    ['order_idx','mov_ver','group'])['subj'].nunique().reset_index()\
                    .rename(columns={'subj':'n_subj'})

# Counts the number of participants that viewed the films in each order and in each version, per group
order_counts_I = np.array(cb_info_full[cb_info_full['group']=='I'].groupby(['order_idx'])['n_subj'].sum())
ver_counts_I = np.array(cb_info_full[cb_info_full['group']=='I'].groupby(['mov_ver'])['n_subj'].sum())
order_counts_D = np.array(cb_info_full[cb_info_full['group']=='D'].groupby(['order_idx'])['n_subj'].sum())
ver_counts_D = np.array(cb_info_full[cb_info_full['group']=='D'].groupby(['mov_ver'])['n_subj'].sum())

# Displays the results
print('Range of participants in orderXverXgroup combinations: {}-{}\n'.format(min(cb_info_full['n_subj']),\
                                                                         max(cb_info_full['n_subj'])))
print('Immediate group - number of participants per order/version:')
print ('  Number of participants who viewed films in orders 1-6: ' + np.array2string(order_counts_I))
print ('  Number of participants who viewed films in versions 1-4: ' + np.array2string(ver_counts_I))
print('\nDelay group - number of participants per order/version:')
print ('  Number of participants who viewed films in orders 1-6: ' + np.array2string(order_counts_D))
print ('  Number of participants who viewed films in versions 1-4: ' + np.array2string(ver_counts_D))

## Single trial analysis - data preparation

Prepares the dataframes required for the single trial analyses and saves in a csv that can later be read by the R brms script

In [ ]:

# Concatenates the immediate and delay groups of all batches, creating one full dataframe
test_I = pd.concat([res['test_df'] for res in immediate_res])
test_D = pd.concat([res['test_df'] for res in delay_res])

# The subject numbers are restarted in each batch for flexibility (e.g. concatenating 
# only dataframes of one group), so updates the subject number to run across all batches
# of both groups, then concatenates both groups into a single dataframe
test_I['subj'] = test_I['subj'] + (test_I['batch']-1)*batch_n_subjects*2
test_D['subj'] = test_D['subj'] + (test_D['batch']-1)*batch_n_subjects*2 + batch_n_subjects
full_test_df = pd.concat((test_I, test_D)).reset_index(drop=True)
    

# Merges information from the test_df with the surprise_dist_df 
# to aggregate the information for the lm analysis in one dataframe
brms_df = full_test_df.astype({'non_foil_q':'int', 'mov_ver':'int'}).merge(surprise_dist_df[['non_foil_q',\
                'mov_ver', 'prev_S_dist', 'next_S_dist']].drop_duplicates(), on=['non_foil_q', 'mov_ver'], 
                validate='many_to_one')

# Adds a column of whether the participant answered 'old' for the brms analysis. For HC analysis the column
# indicates whether the participant answered old with high confidence
brms_df['ansOld'] = brms_df['conf_level']>0
brms_df['HC_ansOld'] = brms_df['conf_level']==2

# Takes only the subset of relevant columns and sets the dypes of some of them, otherwise this causes
# incorrect conversion to an R dataframe. This code is commented out since the notebook crashes when
# trying to run the R code in it. Left here commented so other tests can be run in the noteboko if needed
# brms_df.drop(columns=['target_q', 'batch', 'conf_level', 'mov_ver'], inplace=True)
# brms_df = brms_df.astype({'subj':'int', 'exc_subj':'bool', 'non_foil_q':'int', 'action_idx':'int'})

# Saves as a csv file
brms_df.to_csv(brms_dir + 'brms_df.csv')


## Single trial analysis - model preparation
Creates initial models (the first chain) as well as submission scripts so the full models can be parallelised. Then submits the jobs to the cluster.
As the notebook crashes when running the code, runs an R script called create_brms_models.R that should be placed in the R_dir. Once it has finished, submits the jobs.

The R script is a slight modification of code written by Alex Quent:
https://github.com/JAQuent/bayesianMLM/tree/master/CBU_clusterGuide

___Important:___ 
1. This requires a slurm parallelisation system. If anyone would like to run this and doesn't have slurm, please contact me and I will try to help
2. This part will take a long time, without printing anything to the screen
3. When the cell stops running, it doesn't indicate the R script has finished. The following cell can be used to verify all jobs have finished before proceeding

In [ ]:
# Runs the R script
R_exit_code = os.system('Rscript {}{} {}'.format(R_dir, create_models_fname, brms_dir))

# Submits the jobs created by the script, or displays an error if the R code exited with a non-zero exit code
if (R_exit_code != 0):
    print(f'Rscript exited with non-zero code {R_exit_code}, try running directly from shell to view error')
else:
    submit_scripts = glob.glob(brms_dir + '_rslurm*/submit.sh')
    if (len(submit_scripts)==0):
        print('Rscript did not execute properly, try running directly from shell to view error')
    else:
        for submit_fname in submit_scripts:
            os.system('sbatch --cpus-per-task 8 ' + submit_fname)

## Single trial analysis - test whether jobs have finished
Checks whether the resulting files of the jobs have been created. The cell is set to be run manually, otherwise would require an infinite loop that keeps checking when the files are created and could cause the notebook to crash. Note, if the analysis is rerun for some reason - the files should be deleted before rerunning (otherwise the test below won't work). Once this cell prints that all jobs have finished, it will be possible to proceed to the hypothesis testing that follows

In [ ]:
# Checks first if the scripts have been created
if (len(submit_scripts)==0):
    print ('No scripts have been created yet')

else:
    # For each of the submit scripts created - checks whether a file called results_0.RDS has been created
    all_created = True
    for submit_fname in submit_scripts:
        if (not os.path.exists(os.path.dirname(submit_fname) + '/results_0.RDS')):
            print ('{}/results_0.RDS not yet created'.format(os.path.dirname(submit_fname)))
            all_created = False

    if (all_created):
        print('All jobs have finished - proceed to next step')


## Single trial analysis - test hypotheses
Combines the different chains of each model and tests the relevant hypotheses.

This code is a slight modification of code written by Alex Quent:
https://github.com/JAQuent/bayesianMLM/tree/master/CBU_clusterGuide

In [ ]:
%%R -i brms_dir

# Sets seed to arbitrary value for reproducibility
set.seed(26)

# Results folder prefix
pattern <- '_rslurm_'


# Gets all rslurm folders in the brms directory
allFiles      <- list.files(brms_dir)
rslurmFolders <- allFiles[grepl(pattern, allFiles)]
modelNames    <- gsub(paste0(pattern, '(.+)'), '\\1', rslurmFolders)

# Goes through all rslurm folders
for(j in 1:length(rslurmFolders)){
    
    # Reads the rlsurm results file
    tempList <- readRDS(paste0(brms_dir, '/', rslurmFolders[j], '/results_0.RDS'), refhook = NULL)

    # Combines all runs of the model
    tempModel_args <- list()
    for (k in 1:length(tempList)) {
      tempModel_args[[k]] <- tempList[[k]]$model
    }
    tempModel <- combine_models(mlist=tempModel_args)

    # Assign model name based on folder name
    assign(modelNames[j], tempModel)
}

# Clears variables
rm('tempList')
rm('tempModel')
rm('tempModel_args')

# Saves the combined model to the brms directory
save.image(paste(brms_dir, 'combinedModels.RData', sep="/"))

# Runs three tests equivalent to the analysis on the averaged proactive data, once for regular
# confidence and once for high confidence
# - main effect of surprise
# - surpriseXq-type interaction
# - surpriseXq-typeXgroup interaction
pro_S_bf = bayes_factor(model_pro_w_S, model_pro_no_S)
pro_SQT_bf = bayes_factor(model_pro_no_3way, model_pro_no_SQT)
pro_3way_bf = bayes_factor(model_pro_full, model_pro_no_3way)
HC_pro_S_bf = bayes_factor(model_HC_pro_w_S, model_HC_pro_no_S)
HC_pro_SQT_bf = bayes_factor(model_HC_pro_no_3way, model_HC_pro_no_SQT)
HC_pro_3way_bf = bayes_factor(model_HC_pro_full, model_HC_pro_no_3way)

# Prints the results
print(paste0('Evidence for/against main effect of surprise: ', format(pro_S_bf$bf, digits=2), 
             '/', format(1/pro_S_bf$bf, digits=2)))
print(paste0('Evidence for/against surprise/Q-type interaction: ', format(pro_SQT_bf$bf, digits=2), 
            '/', format(1/pro_SQT_bf$bf, digits=2)))
print(paste0('Evidence for/against 3-way interaction: ', format(pro_3way_bf$bf, digits=2), 
             '/', format(1/pro_3way_bf$bf, digits=2)))
print(paste0('Evidence for/against main effect of surprise (HC): ', format(HC_pro_S_bf$bf, digits=2), 
             '/', format(1/HC_pro_S_bf$bf, digits=2)))
print(paste0('Evidence for/against surprise/Q-type interaction (HC): ', format(HC_pro_SQT_bf$bf, digits=2), 
            '/', format(1/HC_pro_SQT_bf$bf, digits=2)))
print(paste0('Evidence for/against 3-way interaction (HC): ', format(HC_pro_3way_bf$bf, digits=2), 
             '/', format(1/HC_pro_3way_bf$bf, digits=2)))

# Additional statistical tests
Runs several calculations of interest that provide additional information, for a more complete picture, but these are not part of the predefined stats.

## Additional stats (R)
Runs the following calculations (both for Pr and for HC-Pr):
- Tests for a groupXsurprise interaction in the retroactive 3-way anova
- Tests for effects of surprise separated by group (even if there is no current evidence in favour of interactions involving group). These include both a main effect of surprise and an interaction of question-type and surprise within each group

In [ ]:
%%R -i full_subj_avg_df -i full_diff_df

# Sets seed to arbitrary value at the beginning of each R cell, otherwise some results won't be reproducible
set.seed(26)

# Turns the subj column into a factor - will be converted to numeric by default
# Also sets the others to be factors (though they're already strings), since something
# in the conversion in the notebook works differently and at least one isn't recognised 
# as a factor otherwise
full_subj_avg_df[,'subj'] <- as.factor(full_subj_avg_df[,'subj'])
full_subj_avg_df[,'q_type'] <- as.factor(full_subj_avg_df[,'q_type'])
full_subj_avg_df[,'surprise_type'] <- as.factor(full_subj_avg_df[,'surprise_type'])
full_subj_avg_df[,'group'] <- as.factor(full_subj_avg_df[,'group'])

# Separates the retroactive data by group (removing excluded subjects)
retroactive_data_I <- subset(full_subj_avg_df, (q_type=='preS' | q_type=='preT') & 
                             exc_subj=='FALSE' & group=='I')
retroactive_data_D <- subset(full_subj_avg_df, (q_type=='preS' | q_type=='preT') & 
                             exc_subj=='FALSE' & group=='D')

# Runs the following anovas (assumes the full retroactive one has already been run):
# - A 2-way anova with question-type and surprise for the retroactive data of the immediate group
# - A 2-way anova with question-type and surprise for the retroactive data of the delay group
retro_I_bf = anovaBF(Pr ~ q_type*surprise_type + subj, 
                     data=retroactive_data_I, whichRandom='subj')
retro_D_bf = anovaBF(Pr ~ q_type*surprise_type + subj, 
                     data=retroactive_data_D, whichRandom='subj')
retro_HC_I_bf = anovaBF(HC_Pr ~ q_type*surprise_type + subj, 
                        data=retroactive_data_I, whichRandom='subj')
retro_HC_D_bf = anovaBF(HC_Pr ~ q_type*surprise_type + subj, 
                        data=retroactive_data_D, whichRandom='subj')

# Extracts BF in favour of the alternative (H1) for each of the tests described above
bf_retro_group_surprise_H1 = as.numeric(as.vector(retro_bf[17]/retro_bf[15]))
bf_retro_I_main_surprise_H1 = as.numeric(as.vector(retro_I_bf[3]/retro_I_bf[1]))
bf_retro_D_main_surprise_H1 = as.numeric(as.vector(retro_D_bf[3]/retro_D_bf[1]))
bf_retro_I_surprise_q_type_H1 = as.numeric(as.vector(retro_I_bf[4]/retro_I_bf[3]))
bf_retro_D_surprise_q_type_H1 = as.numeric(as.vector(retro_D_bf[4]/retro_D_bf[3]))
bf_HC_retro_group_surprise_H1 = as.numeric(as.vector(retro_HC_bf[17]/retro_HC_bf[15]))
bf_HC_retro_I_main_surprise_H1 = as.numeric(as.vector(retro_HC_I_bf[3]/retro_HC_I_bf[1]))
bf_HC_retro_D_main_surprise_H1 = as.numeric(as.vector(retro_HC_D_bf[3]/retro_HC_D_bf[1]))
bf_HC_retro_I_surprise_q_type_H1 = as.numeric(as.vector(retro_HC_I_bf[4]/retro_HC_I_bf[3]))
bf_HC_retro_D_surprise_q_type_H1 = as.numeric(as.vector(retro_HC_D_bf[4]/retro_HC_D_bf[3]))

# Makes sure the correct models were taken from the anovas (in case the order can change)
add_validated <- TRUE
add_validated <- add_validated & setequal(unlist(strsplit(names(as.vector(retro_bf[15])), ' + ', TRUE)),
                        c('group','q_type','surprise_type', 'group:q_type', 'q_type:surprise_type','subj'))
add_validated <- add_validated & setequal(unlist(strsplit(names(as.vector(retro_I_bf[4])), ' + ', TRUE)), 
                        c('q_type','surprise_type', 'q_type:surprise_type','subj'))
add_validated <- add_validated & setequal(unlist(strsplit(names(as.vector(retro_I_bf[3])), ' + ', TRUE)), 
                        c('q_type','surprise_type','subj'))
add_validated <- add_validated & setequal(unlist(strsplit(names(as.vector(retro_I_bf[1])), ' + ', TRUE)), 
                        c('q_type','subj'))

## Additional stats - display results

In [ ]:
# Pulls the relevant variables from the R code and converts to python native data types

%Rpull add_validated bf_retro_group_surprise_H1 bf_retro_I_main_surprise_H1 bf_retro_D_main_surprise_H1 \
bf_retro_I_surprise_q_type_H1 bf_retro_D_surprise_q_type_H1 bf_HC_retro_group_surprise_H1 \
bf_HC_retro_I_main_surprise_H1 bf_HC_retro_D_main_surprise_H1 bf_HC_retro_I_surprise_q_type_H1 \
bf_HC_retro_D_surprise_q_type_H1

add_validated = bool(np.asarray(add_validated)[0])
bf_retro_group_surprise_H1 = np.asarray(bf_retro_group_surprise_H1)[0]
bf_retro_I_main_surprise_H1 = np.asarray(bf_retro_I_main_surprise_H1)[0]
bf_retro_D_main_surprise_H1 = np.asarray(bf_retro_D_main_surprise_H1)[0]
bf_retro_I_surprise_q_type_H1 = np.asarray(bf_retro_I_surprise_q_type_H1)[0]
bf_retro_D_surprise_q_type_H1 = np.asarray(bf_retro_D_surprise_q_type_H1)[0]
bf_HC_retro_group_surprise_H1 = np.asarray(bf_HC_retro_group_surprise_H1)[0]
bf_HC_retro_I_main_surprise_H1 = np.asarray(bf_HC_retro_I_main_surprise_H1)[0]
bf_HC_retro_D_main_surprise_H1 = np.asarray(bf_HC_retro_D_main_surprise_H1)[0]
bf_HC_retro_I_surprise_q_type_H1 = np.asarray(bf_HC_retro_I_surprise_q_type_H1)[0]
bf_HC_retro_D_surprise_q_type_H1 = np.asarray(bf_HC_retro_D_surprise_q_type_H1)[0]


# Prints results of calculations, first verifying the chosen models are the correct ones
if (not add_validated):
    print('The ANOVA models chosen for the calculations do not match the expected ones.' + \
         'This may be due to different platforms/versions of the BayesFactor package.' + \
          'View cell above to identify the correct models from the bf variable (that includes)' + \
          'all models in the anova')
else:
    print('Results of additional tests, all retroactive (H1/H0):')
    print('   Evidence of a groupXsurprise interaction: '\
              + '(%.3g/%.3g)'% (bf_retro_group_surprise_H1, 1/bf_retro_group_surprise_H1))
    print('   Evidence of a main effect of surprise (immediate group): '\
              + '(%.3g/%.3g)'% (bf_retro_I_main_surprise_H1, 1/bf_retro_I_main_surprise_H1))
    print('   Evidence of a main effect of surprise (delay group): '\
              + '(%.3g/%.3g)'% (bf_retro_D_main_surprise_H1, 1/bf_retro_D_main_surprise_H1))
    print('   Evidence of a q-typeXsurprise interaction (immediate group): '\
              + '(%.3g/%.3g)'% (bf_retro_I_surprise_q_type_H1, 1/bf_retro_I_surprise_q_type_H1))
    print('   Evidence of a q-typeXsurprise interaction (delay group): '\
              + '(%.3g/%.3g)'% (bf_retro_D_surprise_q_type_H1, 1/bf_retro_D_surprise_q_type_H1))
    print('\nResults of additional tests, all retroactive - HC Pr (H1/H0):')
    print('   Evidence of a groupXsurprise interaction: '\
              + '(%.3g/%.3g)'% (bf_HC_retro_group_surprise_H1, 1/bf_HC_retro_group_surprise_H1))
    print('   Evidence of a main effect of surprise (immediate group): '\
              + '(%.3g/%.3g)'% (bf_HC_retro_I_main_surprise_H1, 1/bf_HC_retro_I_main_surprise_H1))
    print('   Evidence of a main effect of surprise (delay group): '\
              + '(%.3g/%.3g)'% (bf_HC_retro_D_main_surprise_H1, 1/bf_HC_retro_D_main_surprise_H1))
    print('   Evidence of a q-typeXsurprise interaction (immediate group): '\
              + '(%.3g/%.3g)'% (bf_HC_retro_I_surprise_q_type_H1, 1/bf_HC_retro_I_surprise_q_type_H1))
    print('   Evidence of a q-typeXsurprise interaction (delay group): '\
              + '(%.3g/%.3g)'% (bf_HC_retro_D_surprise_q_type_H1, 1/bf_HC_retro_D_surprise_q_type_H1))
    

# Sanity checks
Runs various tests to validate the results, including testing that the dataframes have the expected number of entries and that the Bayesian stats are comparable to frequentists stats

## Makes sure dataframes include the expected number of entries
Most sanity checks are within the process_batch function, but since aggregation across batches is done in the notebook, adds a few tests here

In [ ]:
# For each of the dataframes (full_subj_avg_df full_diff_df first_batch_df target_diff_df), makes sure
# the following are as expected (in this order):
#  - list of unique subjects
#  - number of entries per subject

df_verifiied = True

n_q_type = 4
n_surprise_type = 2
total_n_subj = batch_num*2*batch_n_subjects
first_batch_n_subj = 2*batch_n_subjects

# Counts the number of question pairs (foil + non-foil counted once)
n_q_pairs = len(np.unique(immediate_res[0]['q_info_df']['non_foil_q']))

# full_subj_avg_df verification
df_verifiied = df_verifiied and np.array_equal(np.unique(full_subj_avg_df['subj']),np.arange(total_n_subj))
df_verifiied = df_verifiied and (full_subj_avg_df.groupby(['subj', 'q_type', 'surprise_type']).ngroups \
                                == total_n_subj*n_q_type*n_surprise_type)

# full_diff_df verification
df_verifiied = df_verifiied and np.array_equal(np.unique(full_diff_df['subj']),np.arange(total_n_subj))
df_verifiied = df_verifiied and (full_diff_df.groupby(['subj', 'q_type']).ngroups \
                                == total_n_subj*n_q_type)

# first_batch_df verification
df_verifiied = df_verifiied and np.array_equal(np.unique(first_batch_df['subj']),np.arange(first_batch_n_subj))
df_verifiied = df_verifiied and (first_batch_df.groupby(['subj', 'q_type', 'surprise_type']).ngroups \
                                == first_batch_n_subj*n_q_type*n_surprise_type)

# target_diff_df verification
df_verifiied = df_verifiied and np.array_equal(np.unique(target_diff_df['subj']),np.arange(first_batch_n_subj))
df_verifiied = df_verifiied and (target_diff_df.groupby(['subj', 'q_type']).ngroups \
                                == first_batch_n_subj)


# brms_df verification
df_verifiied = df_verifiied and np.array_equal(np.unique(brms_df['subj']),np.arange(total_n_subj))
df_verifiied = df_verifiied and (brms_df.groupby(['subj', 'non_foil_q']).ngroups \
                                == total_n_subj*n_q_pairs)

if (not df_verifiied):
    print ('Problem in dataframe verification')
else:
    print ('Sanity checks ok')

## Verifies surprise>neutral memory

Verifies that there is better memory for surprising targets relative to neutral ones. This was verified for the first batch, here makes sure the effect remains for the entire group of participants

In [ ]:
%%R -i full_diff_df

# Sets seed to arbitrary value at the beginning of each R cell, otherwise some results won't be reproducible
set.seed(26)

# Turns the subj column into a factor - will be converted to numeric by default
# Also sets the others to be factors (though they're already strings), since something
# in the conversion in the notebook works differently and at least one isn't recognised 
# as a factor otherwise
full_diff_df[,'subj'] <- as.factor(full_diff_df[,'subj'])
full_diff_df[,'q_type'] <- as.factor(full_diff_df[,'q_type'])
full_diff_df[,'group'] <- as.factor(full_diff_df[,'group'])

# Slices the data to have the relevant subsets for each test

T_I_diff <- subset(full_diff_df, q_type=='T' & group=='I' & exc_subj=='FALSE')
T_D_diff <- subset(full_diff_df, q_type=='T' & group=='D' & exc_subj=='FALSE')
T_comb_diff <- subset(full_diff_df, q_type=='T' & exc_subj=='FALSE')


# Runs the planned t-tests, looking at surprise effects within each question type

T_I_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(T_I_diff['Pr_diff'])))))
T_D_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(T_D_diff['Pr_diff'])))))
T_comb_bf_H1 <- as.numeric(as.vector(ttestBF(as.numeric(unlist(T_comb_diff['Pr_diff'])))))

cat('Evidence in favour of better memory for surprising targets:\n')
cat(sprintf('  Immediate group: %.2e\n', T_I_bf_H1))
cat(sprintf('  Delay group: %.2e\n', T_D_bf_H1))
cat(sprintf('  Combined %.2e\n', T_comb_bf_H1))


# Exploratory Analysis - dependency

Runs an exploratory analysis (not included in Stage 1 of the registered report) to test whether surprising actions segment the episode, serving as boundaries. To do so, tests the dependency between the different question types - checking whether the dependency changes for surprising events vs. neutral ones

## Exploratory analysis - data preparation

Prepares the data for the dependency analysis - creates a dataframe in which the hits of all questions from the same scene are in one row

In [ ]:
# Pivots table to create a separate column for each question type, then renames the columns 
# (merging into one level of columns) and resets index. Takes only non-foils for this analysis
dep_df = brms_df[~(brms_df['foil'].astype(bool))].rename(columns={'ansOld':'hit', 'HC_ansOld': 'HC_hit'})\
            .pivot_table(index=['subj', 'group', 'exc_subj', 'target_q', 'surprise_type'], 
                                           columns='q_type', values=['hit', 'HC_hit'])

dep_df.columns = list(map("_".join, dep_df.columns))
dep_df.reset_index(inplace=True)

# Divides the dependency df by surprise - creating two dataframes
dep_df_S = dep_df[(dep_df['surprise_type']=='S')].drop(columns='surprise_type')
dep_df_N = dep_df[(dep_df['surprise_type']=='N')].drop(columns='surprise_type')

# Calculates the average of each column and adds back to the dependency dfs so these can be added
# as predictors, separating the average subject memory from the per-question effects which
# demonstrate whether there's dependency
dep_df_S_avg = dep_df_S.groupby(['subj', 'group', 'exc_subj']).mean().drop(columns='target_q')
dep_df_S_avg.columns = ['avg_' + c for c in dep_df_S_avg.columns]
dep_df_N_avg = dep_df_N.groupby(['subj', 'group', 'exc_subj']).mean().drop(columns='target_q')
dep_df_N_avg.columns = ['avg_' + c for c in dep_df_N_avg.columns]
dep_df_S = dep_df_S.merge(dep_df_S_avg.reset_index(), on=['subj', 'group', 'exc_subj'])
dep_df_N = dep_df_N.merge(dep_df_N_avg.reset_index(), on=['subj', 'group', 'exc_subj'])

In [ ]:
# Displayes the distribution of average hit rates for the surprising targets to estimate whether it is
# meaningful to include targets in the dependency analysis.

all_hit_rate = full_subj_avg_df[(full_subj_avg_df['q_type']=='T')&(full_subj_avg_df['surprise_type']=='S')&\
                          (~full_subj_avg_df['exc_subj'])]['hit_rate']
hit_rates, hr_n_subj = np.unique(all_hit_rate, return_counts=True)
hr_df = pd.DataFrame.from_dict({'hit_rate':hit_rates, 'n_subj':hr_n_subj})
hr_df


## Exploratory analysis - dependency calculation (R)

Calculates the dependency using general linear mixed models - to calculate the dependency between hits in one question type (qt1) and another (qt2), calculates the coefficients of qt2 in a model that has qt1 as a DV and the other question types as predictors (all binary - hit/miss per question per participant). Compares this to models in which the pairing of qt2 to qt1 has been randomly shuffled per participant. 

Runs this separately for neutral and surprising scenes to assess whether surprise reduces the dependency. The main question of interest is whether a surprising event affects dependency between preT and postT questions, that straddle the surprise, and as a control tests preS and preT (for completeness calculates dependency for all pairs of question types, excluding targets)

___Important:___ 
1. This requires a slurm parallelisation system. If anyone would like to run this and doesn't have slurm, please contact me and I will try to help
2. This part will take a long time, without printing anything to the screen

In [ ]:
%%R -i dep_df_S -i dep_df_N -i n_nodes -i cpus_per_node -i n_iter

# Sets seed for reproducibility
set.seed(26)

# Turns columns into factors
dep_df_S[,'subj'] <- as.factor(dep_df_S[,'subj'])
dep_df_S[,'group'] <- as.factor(dep_df_S[,'group'])
dep_df_S[,'target_q'] <- as.factor(dep_df_S[,'target_q'])
dep_df_N[,'subj'] <- as.factor(dep_df_N[,'subj'])
dep_df_N[,'group'] <- as.factor(dep_df_N[,'group'])
dep_df_N[,'target_q'] <- as.factor(dep_df_N[,'target_q'])

# Gets separate subsets of the surprising/neutral data
dep_df_S <- subset(dep_df_S, exc_subj=='FALSE')
dep_df_N <- subset(dep_df_N, exc_subj=='FALSE')

# Defines the names of the predictors for each question type
q_types <- c('hit_preS', 'hit_preT', 'hit_postT')
HC_q_types <- c('HC_hit_preS', 'HC_hit_preT', 'HC_hit_postT')
n_q_types <- length(q_types)

# Variables to hold coefficients of the intact models such that array in locations 
# [qt1, qt2] will hold the coefficient of qt2 when modeling qt1
intact_coef_S <- array(NaN, dim=c(n_q_types,n_q_types))
intact_coef_N <- array(NaN, dim=c(n_q_types,n_q_types))
intact_coef_HC_S <- array(NaN, dim=c(n_q_types,n_q_types))
intact_coef_HC_N <- array(NaN, dim=c(n_q_types,n_q_types))

for (qt1 in 1:n_q_types) {

    # Calculates the intact models
    qt1_S_model <- glmer(as.formula(paste(q_types[qt1], ' ~ ', paste(q_types[setdiff(seq(1,n_q_types), 
                qt1)], collapse=' + '), ' + group + (1|subj) + (1|target_q)')), 
                data=dep_df_S, family=binomial, control=glmerControl(optimizer='bobyqa', 
                optCtrl=list(maxfun=2e5)))
    qt1_N_model <- glmer(as.formula(paste(q_types[qt1], ' ~ ', paste(q_types[setdiff(seq(1,n_q_types), 
                qt1)], collapse=' + '), ' + group + (1|subj) + (1|target_q)')), 
                data=dep_df_N, family=binomial, control=glmerControl(optimizer='bobyqa', 
                optCtrl=list(maxfun=2e5)))

    # Same for HC
    HC_qt1_S_model <- glmer(as.formula(paste(HC_q_types[qt1], ' ~ ', paste(HC_q_types[setdiff(seq(1,n_q_types), 
                qt1)], collapse=' + '), ' + group + (1|subj) + (1|target_q)')), 
                data=dep_df_S, family=binomial, control=glmerControl(optimizer='bobyqa', 
                optCtrl=list(maxfun=2e5)))
    HC_qt1_N_model <- glmer(as.formula(paste(HC_q_types[qt1], ' ~ ', paste(HC_q_types[setdiff(seq(1,n_q_types), 
                qt1)], collapse=' + '), ' + group + (1|subj) + (1|target_q)')), 
                data=dep_df_N, family=binomial, control=glmerControl(optimizer='bobyqa', 
                optCtrl=list(maxfun=2e5)))

    # Iterates over the other predictors and extracts the coefficients
    for (qt2 in setdiff(1:length(q_types), qt1)) {
        intact_coef_S[qt1,qt2] <- as.numeric(fixef(qt1_S_model)[grep(q_types[qt2], names(fixef(qt1_S_model)))])
        intact_coef_N[qt1,qt2] <- as.numeric(fixef(qt1_N_model)[grep(q_types[qt2], names(fixef(qt1_N_model)))])
        intact_coef_HC_S[qt1,qt2] <- as.numeric(fixef(HC_qt1_S_model)[grep(HC_q_types[qt2], names(fixef(HC_qt1_S_model)))])
        intact_coef_HC_N[qt1,qt2] <- as.numeric(fixef(HC_qt1_N_model)[grep(HC_q_types[qt2], names(fixef(HC_qt1_N_model)))])
    }
}


# Creates two variables with the parameters for the parallelisation - first pairs of 
# question-types (qt1 will be the DV and qt2 will be the shuffled predictor), then random
# seeds. Uses the same seed for each of the 6 pairs (though this isn't necessary)
qt_pairs <- combn(seq(n_q_types),2)
qt_pairs <- cbind(qt_pairs, qt_pairs[c(2,1),])
n_pairs <- ncol(qt_pairs)
iter_idx <- as.vector(t(replicate(n_pairs,seq(n_iter))))
qt_pairs <- qt_pairs[,rep(seq(n_pairs), n_iter)]
rnd_seeds <- sample(99999, n_iter)
rnd_seeds <- as.vector(t(replicate(n_pairs,rnd_seeds)))

calc_glmer <- function(i, qt1, qt2, seed) {
  
  # Sets for reproducibility
  set.seed(seed)
  
  # Creates versions of the dataframes in which qt2 columns are shuffled within subject
  rnd_S <- ddply(dep_df_S[,c('subj', q_types[qt2], HC_q_types[qt2])], .(subj), 
                 function(x) x[sample(nrow(x),nrow(x)),])
  rnd_N <- ddply(dep_df_N[,c('subj', q_types[qt2], HC_q_types[qt2])], .(subj), 
                 function(x) x[sample(nrow(x),nrow(x)),])
  
  # Creates versions of the dataframes in which qt2 has been shuffled. Modifies regular and HC in the same DF,
  # though these will be used in separate models
  curr_qt2_rnd_S <- dep_df_S
  curr_qt2_rnd_S[,q_types[qt2]] <- rnd_S[,q_types[qt2]]
  curr_qt2_rnd_S[,HC_q_types[qt2]] <- rnd_S[,HC_q_types[qt2]]
  curr_qt2_rnd_N <- dep_df_N
  curr_qt2_rnd_N[,q_types[qt2]] <- rnd_N[,q_types[qt2]]
  curr_qt2_rnd_N[,HC_q_types[qt2]] <- rnd_N[,HC_q_types[qt2]]
  
  # Runs an glmer with qt1 as the dependent variable and qt2 shuffled, once for surprising and once for neutral
  # scenes. Then runs the same for HC
  qt2_rnd_S_model <- glmer(as.formula(paste(q_types[qt1], ' ~ ', paste(q_types[setdiff(seq(1,length(q_types)), 
                           qt1)], collapse=' + '), ' + group + (1|subj) + (1|target_q)')), 
                           data=curr_qt2_rnd_S, family=binomial, control=glmerControl(optimizer='bobyqa',
                           optCtrl=list(maxfun=2e5)))
  qt2_rnd_N_model <- glmer(as.formula(paste(q_types[qt1], ' ~ ', paste(q_types[setdiff(seq(1,length(q_types)),
                           qt1)], collapse=' + '), ' + group + (1|subj) + (1|target_q)')), 
                           data=curr_qt2_rnd_N, family=binomial, 
                           control=glmerControl(optimizer='bobyqa', optCtrl=list(maxfun=2e5)))
  HC_qt2_rnd_S_model <- glmer(as.formula(paste(HC_q_types[qt1], ' ~ ', 
                            paste(HC_q_types[setdiff(seq(1,length(q_types)), qt1)], collapse=' + '),
                            ' +  group + (1|subj) + (1|target_q)')), data=curr_qt2_rnd_S, 
                            family=binomial, control=glmerControl(optimizer='bobyqa', 
                            optCtrl=list(maxfun=2e5)))
  HC_qt2_rnd_N_model <- glmer(as.formula(paste(HC_q_types[qt1], ' ~ ', 
                            paste(HC_q_types[setdiff(seq(1,length(q_types)), qt1)], collapse=' + '),
                            ' + group + (1|subj) + (1|target_q)')), data=curr_qt2_rnd_N, 
                            family=binomial, control=glmerControl(optimizer='bobyqa',
                            optCtrl=list(maxfun=2e5)))

  # Saves coefficients
  coef_S <- as.numeric(fixef(qt2_rnd_S_model)[grep(q_types[qt2], names(fixef(qt2_rnd_S_model)))])
  coef_N <- as.numeric(fixef(qt2_rnd_N_model)[grep(q_types[qt2], names(fixef(qt2_rnd_N_model)))])
  coef_HC_S <- as.numeric(fixef(HC_qt2_rnd_S_model)[grep(HC_q_types[qt2], names(fixef(HC_qt2_rnd_S_model)))])
  coef_HC_N <- as.numeric(fixef(HC_qt2_rnd_N_model)[grep(HC_q_types[qt2], names(fixef(HC_qt2_rnd_N_model)))])

  c(i=i, qt_dep=qt1, qt_rnd=qt2, coef_S=coef_S, coef_N=coef_N, coef_HC_S=coef_HC_S, coef_HC_N=coef_HC_N)
}

sjob <- slurm_apply(calc_glmer, data.frame(i=iter_idx, qt1=qt_pairs[1,], qt2=qt_pairs[2,], seed=rnd_seeds),
                    jobname = 'calc_glmer',
                    add_objects = c('q_types', 'HC_q_types', 'n_q_types', 'dep_df_S', 'dep_df_N'),
                    nodes = n_nodes, cpus_per_node = cpus_per_node, submit = TRUE)
all_coefs <- get_slurm_out(sjob, outtype = 'table')

# Converts to matrices to make it simpler to convert to python
all_coef_S <- array(NaN, dim=c(n_iter,n_q_types,n_q_types))
all_coef_N <- array(NaN, dim=c(n_iter,n_q_types,n_q_types))
all_coef_HC_S <- array(NaN, dim=c(n_iter,n_q_types,n_q_types))
all_coef_HC_N <- array(NaN, dim=c(n_iter,n_q_types,n_q_types))
for (j in 1:n_iter) {
  for (qt1 in 1:(n_q_types-1)) {
    for (qt2 in (qt1+1):n_q_types) {
      all_coef_S[j,qt1,qt2] = subset(all_coefs, qt_dep==qt1 & qt_rnd==qt2 & i==j)[,'coef_S']
      all_coef_N[j,qt1,qt2] = subset(all_coefs, qt_dep==qt1 & qt_rnd==qt2 & i==j)[,'coef_N']
      all_coef_HC_S[j,qt1,qt2] = subset(all_coefs, qt_dep==qt1 & qt_rnd==qt2 & i==j)[,'coef_HC_S']
      all_coef_HC_N[j,qt1,qt2] = subset(all_coefs, qt_dep==qt1 & qt_rnd==qt2 & i==j)[,'coef_HC_N']
      all_coef_S[j,qt2,qt1] = subset(all_coefs, qt_dep==qt2 & qt_rnd==qt1 & i==j)[,'coef_S']
      all_coef_N[j,qt2,qt1] = subset(all_coefs, qt_dep==qt2 & qt_rnd==qt1 & i==j)[,'coef_N']
      all_coef_HC_S[j,qt2,qt1] = subset(all_coefs, qt_dep==qt2 & qt_rnd==qt1 & i==j)[,'coef_HC_S']
      all_coef_HC_N[j,qt2,qt1] = subset(all_coefs, qt_dep==qt2 & qt_rnd==qt1 & i==j)[,'coef_HC_N']
    }
  }
}      

In [ ]:
# Pulls the relevant variables from the R code and converts to python native data types

%Rpull all_coef_S all_coef_N all_coef_HC_S all_coef_HC_N q_types HC_q_types intact_coef_S intact_coef_N \
intact_coef_HC_S intact_coef_HC_N

# Saves directional dependency as arrays
intact_coef_S = np.asarray(intact_coef_S) 
intact_coef_N = np.asarray(intact_coef_N)
intact_coef_HC_S = np.asarray(intact_coef_HC_S)
intact_coef_HC_N = np.asarray(intact_coef_HC_N) 

all_coef_S = np.asarray(all_coef_S)
all_coef_N = np.asarray(all_coef_N) 
all_coef_HC_S = np.asarray(all_coef_HC_S) 
all_coef_HC_N = np.asarray(all_coef_HC_N) 

# Converts to bidirectional dependency - for each pair of question types, averages the dependency
# in the two directions
intact_coef_S_bi = (intact_coef_S + np.transpose(intact_coef_S))/2
intact_coef_N_bi = (intact_coef_N + np.transpose(intact_coef_N))/2
intact_coef_HC_S_bi = (intact_coef_HC_S + np.transpose(intact_coef_HC_S))/2
intact_coef_HC_N_bi = (intact_coef_HC_N + np.transpose(intact_coef_HC_N))/2

all_coef_S_bi = (all_coef_S + np.transpose(all_coef_S, (0,2,1)))/2
all_coef_N_bi = (all_coef_N + np.transpose(all_coef_N, (0,2,1)))/2
all_coef_HC_S_bi = (all_coef_HC_S + np.transpose(all_coef_HC_S, (0,2,1)))/2
all_coef_HC_N_bi = (all_coef_HC_N + np.transpose(all_coef_HC_N, (0,2,1)))/2

# For each q-type pair - calculates the difference between S and N in the intact and each iteration
intact_coef_diff = intact_coef_N - intact_coef_S
intact_coef_HC_diff = intact_coef_HC_N - intact_coef_HC_S
all_coef_diff = all_coef_N - all_coef_S
all_coef_HC_diff = all_coef_HC_N - all_coef_HC_S

intact_coef_diff_bi = intact_coef_N_bi - intact_coef_S_bi
intact_coef_HC_diff_bi = intact_coef_HC_N_bi - intact_coef_HC_S_bi
all_coef_diff_bi = all_coef_N_bi - all_coef_S_bi
all_coef_HC_diff_bi = all_coef_HC_N_bi - all_coef_HC_S_bi



## Dependency analysis - results
Plots the results (Figure 3 of the paper)

In [ ]:
# Defines colors for plots

plot_color = (0.00392157, 0.31372549, 0.40392157)
line_plot_color =  (0.00392157, 0.31372549, 0.40392157)

plot_color_N = (0.12156862745098039, 0.4666666666666667, 0.7058823529411765)
plot_color_S = (1.0, 0.4980392156862745, 0.054901960784313725)


# panel A - preT-postT depenedency, separately for neutral and surprising events
fig, ax = plt.subplots(figsize=(fig_w, fig_h), dpi=plot_dpi)
sns.distplot(all_coef_N_bi[:,1,2], ax=ax, kde=False, color=plot_color_N)
plt.axvline(intact_coef_N_bi[1,2], color=plot_color_N, linewidth=10)
sns.distplot(all_coef_S_bi[:,1,2], ax=ax, kde=False, color=plot_color_S)
plt.axvline(intact_coef_S_bi[1,2], color=plot_color_S, linewidth=10)
plt.xlabel('Dependency', fontsize=label_font)
plt.ylabel('Num iterations', fontsize=label_font)
plt.yticks(fontsize=tick_font)
plt.xticks(fontsize=tick_font)
if (save_figs):
    fig.savefig(f'{fig_dir}dep_N_S_preT_postT.pdf')

# panel B - plot of preT-postT dependency difference (N-S)
fig, ax = plt.subplots(figsize=(fig_w, fig_h), dpi=plot_dpi)
sns.distplot(all_coef_diff_bi[:,1,2], ax=ax, kde=False, color=plot_color)
plt.axvline(intact_coef_diff_bi[1,2], color=line_plot_color, linewidth=10)
plt.xlabel('Neutral/surprise dependency difference', fontsize=label_font)
plt.ylabel('Num iterations', fontsize=label_font)
plt.yticks(fontsize=tick_font)
plt.xticks(fontsize=tick_font)
if (save_figs):
    fig.savefig(f'{fig_dir}dep_diff_preT_postT.pdf')

# panel C - plot of preS-preT dependency difference (N-S)
fig, ax = plt.subplots(figsize=(fig_w, fig_h), dpi=plot_dpi)
sns.distplot(all_coef_diff_bi[:,0,1], ax=ax, kde=False, color=plot_color)
plt.axvline(intact_coef_diff_bi[0,1], color=line_plot_color, linewidth=10)
plt.xlabel('Neutral/surprise dependency difference', fontsize=label_font)
plt.ylabel('Num iterations', fontsize=label_font)
plt.yticks(fontsize=tick_font)
plt.xticks(fontsize=tick_font)
if (save_figs):
    fig.savefig(f'{fig_dir}dep_diff_preS_preT.pdf')

### Calculates the significance of the dependency for each of the relevant comparisons

In [ ]:
# Calculates the p-value for the neutral/surprise dependency for each action type pair
preS_preT_N_pval = np.mean(np.greater(abs(all_coef_N_bi[:,0,1]), abs(intact_coef_N_bi[0,1])),0)
preT_postT_N_pval = np.mean(np.greater(abs(all_coef_N_bi[:,1,2]), abs(intact_coef_N_bi[1,2])),0)
preS_preT_S_pval = np.mean(np.greater(abs(all_coef_S_bi[:,0,1]), abs(intact_coef_S_bi[0,1])),0)
preT_postT_S_pval = np.mean(np.greater(abs(all_coef_S_bi[:,1,2]), abs(intact_coef_S_bi[1,2])),0)

# Calculates the p-values for the neutral-surprise dependency difference
preS_preT_diff_pval = np.mean(np.greater(abs(all_coef_diff_bi[:,0,1]), abs(intact_coef_diff_bi[0,1])),0)
preT_postT_diff_pval = np.mean(np.greater(abs(all_coef_diff_bi[:,1,2]), abs(intact_coef_diff_bi[1,2])),0)

# Compares the preT-postT dependency difference to preS-preT 
preT_postT_v_preS_preT_pval = np.mean(np.greater(abs(all_coef_diff_bi[:,1,2]-all_coef_diff_bi[:,0,1]), abs(intact_coef_diff_bi[1,2]-intact_coef_diff_bi[0,1])),0)

# Prints results to screen
print ('Results of dependency analysis:')
print('   Neutral-version preT/postT dependency: %.3g'% preT_postT_N_pval)
print('   Surprise-version preT/postT dependency: %.3g'% preT_postT_S_pval)
print('   Surprise effect (neutral/surprise diff) on preT/postT dependency: %.3g'% preT_postT_diff_pval)
print('\n   Neutral-version preT/preS dependency: %.3g'% preS_preT_N_pval)
print('   Surprise-version preT/preS dependency: %.3g'% preS_preT_S_pval)
print('   Surprise effect (neutral/surprise diff) on preT/preS dependency: %.3g'% preS_preT_diff_pval)
print('\n   Difference in surprise effects: %.3g'% preT_postT_v_preS_preT_pval)

# Exploratory Analysis - subjective segmentation

Runs an exploratory analysis (not included in Stage 1 of the registered report) testing whether surprise constitutes a subjective event boundary. Plots the boundaries identified by participants along with the objective scene changes and the timings of the surprising targets. Supplementary Figure 4.

In [ ]:
subj_S_boundaries = seg_S_res['subj_boundaries'] 
for m in np.arange(exp_params['n_mov']):
    fig = plt.figure(figsize=(fig_w, fig_h), dpi=plot_dpi)
    ax = plt.gca()
    boundary_subset = subj_S_boundaries[subj_S_boundaries['mov_num']==m+1]
    sns.swarmplot(x=boundary_subset['time']/1000, 
                  hue=boundary_subset['sub'], data=boundary_subset, ax=ax)
    for t in np.array(seg_params['target_actions'][m])/1000:        
        plt.plot([t, t], ax.get_ylim(), color=(0.7,0,0), linewidth=2.0, zorder=0)
    for b in np.array(seg_params['mov_scene_change'][m])/1000:        
        plt.plot([b, b], ax.get_ylim(), color='k', linewidth=2.0, zorder=0) 
    plt.xticks(fontweight='bold', fontsize=10)
    ax.set_xlabel('Time (sec)', {'fontweight': 'bold', 'fontsize' : 14})
    ax.set_title('Film #{}'.format(m+1), {'fontweight': 'bold', 
                 'fontsize' : 14})
    plt.show()
    if (save_figs):
        fig.savefig(f'{fig_dir}boundary_target_match_S_mov{{}}.eps'.format(m+1))
    
    

# Supplementary Analyses
Code for creating Supp. Figures 1-2 - plotting boundaries identified by participants watching the neutral version of the film relative to objective scene changes. Data for Supplementary tables calculated as part of the main analyses.

In [ ]:
# Creates a plot with the scene changes and all boundaries indicated by
# by all subjects (here without the target actions as they are all neutral)
subj_boundaries = seg_res['subj_boundaries'] 
for m in np.arange(exp_params['n_mov']):
    fig = plt.figure(figsize=(fig_w, fig_h), dpi=plot_dpi)
    ax = plt.gca()
    boundary_subset = subj_boundaries[subj_boundaries['mov_num']==m+1]
    sns.swarmplot(x=boundary_subset['time']/1000, 
                  hue=boundary_subset['sub'], data=boundary_subset, ax=ax)
    for b in np.array(seg_params['mov_scene_change'][m])/1000:        
        plt.plot([b, b], ax.get_ylim(), color='k', linewidth=2.0, zorder=0) 
    plt.xticks(fontweight='bold', fontsize=tick_font)
    ax.set_xlabel('Time (sec)', {'fontweight': 'bold', 'fontsize' : label_font})
    ax.set_title('Film #{}'.format(m+1), {'fontweight': 'bold', 
                 'fontsize' : label_font})
    plt.show()
    if (save_figs):
        fig.savefig(f'{fig_dir}boundary_match_mov{{}}.eps'.format(m+1))
    

# Plots all peak heights, to verify the distribution is divided into two clear 
# clusters, and the identification of boundaries isn't dependent on the 
# precise threshold chosen (of the minimal number of participants that ) 
all_peak_heights = seg_res['all_peak_heights']
fig = plt.figure(figsize=(fig_w, fig_h), dpi=plot_dpi)
ax = plt.gca()
sns.swarmplot(x=np.ones(len(all_peak_heights)), y=all_peak_heights)
plt.xticks([])
plt.yticks([5,10,15], fontweight='bold', fontsize=tick_font)
ax.set_ylabel('#participants', {'fontweight': 'bold', 'fontsize' : label_font})
if (save_figs):
        fig.savefig(f'{fig_dir}boundary_peaks.eps')

# Prints the average distance between the marked boundaries and the scene
# changes, the maximal distance and the number of scene changes that
# weren't identified by subjects.
subj_mean_boundaries = seg_res['subj_mean_boundaries']
n_no_match = sum(subj_mean_boundaries['time'].isna())
mean_delay = np.mean(subj_mean_boundaries['scene_dist'])
max_dist = np.max(abs(subj_mean_boundaries['scene_dist']))
print('Number of unidentified scenes: %.3g'% n_no_match)
print('Mean delay between scene change and boundary: %.3g'% mean_delay)
print('Max (abs) distance between scene change and boundary: %.3g'% max_dist)